In [1]:
import os
os.getpid()

17664

In [2]:
import matplotlib
%matplotlib inline
import seaborn
seaborn.set_style("darkgrid")
import matplotlib.pyplot as plt
from matplotlib import colors

import pandas as pd
idx = pd.IndexSlice
pd.options.display.max_columns = 35
from xlsxwriter.utility import xl_col_to_name, xl_range, xl_rowcol_to_cell
import numpy as np
import os.path
from pathlib import Path
import glob
import re
import lxml.etree as etree
#import xml.etree.ElementTree as ET
import json
import scipy
from math import log2
from statsmodels.sandbox.stats.multicomp import multipletests

# Use autoreload to keep up-to-date version of custom packages

import mwTools
import cterminal
import analysis_COG_enrichment
from cterminal import full_analysis_multispecies
from cterminal import simplify_prefix
from analysis_COG_enrichment import enrichment_analysis_cterminal_subset
from mwTools.paths import p

p = p()
print(p.rootPath)
rootPath = p.rootPath
cterminalPath = p.cterminalPath
refSeqPath = p.refSeqPath
taxonomyPath = p.taxonomyPath
phylaPath = p.phylaPath
entrezGenePath = p.entrezGenePath
OMAPath = p.OMAPath
eggNOGPath = p.eggNOGPath
NCBI_COG_path = p.NCBI_COG_path
analysisEggnogPath = p.analysisEggnogPath
analysisCtermDataPath = p.analysisCtermDataPath
analysisCtermPlotsPath = p.analysisCtermPlotsPath

from cterminal import aaTable
print(len(aaTable))

/users/lserrano/mweber
/users/lserrano/mweber
20


In [3]:
OMAPath

PosixPath('/users/lserrano/mweber/Databases/OMA')

# EggNOG database

### taxonomy table EggNOG

In [4]:
eggTaxonomicDf = pd.read_csv(str(eggNOGPath / 'eggnog4.taxonomic_levels.txt'), sep='\t', header=0)
eggTaxonomicDf.head()

,#tax id,nog prefix,level name,species count,core species count,periphery species count,adherent species count,members tax ids
0,1,NOG,LUCA,3686,848,1183,1655,394 882 883 1140 1148 2110 2850 3055 3218 3694...
1,57723,aciNOG,Acidobacteria,7,6,1,0,204669 234267 240015 401053 682795 940615 981222
2,204432,acidNOG,Acidobacteriia,4,3,1,0,240015 401053 682795 940615
3,422676,acoNOG,Aconoidasida,8,1,7,0,5821 5825 5833 5850 5855 333668 352914 353154
4,201174,actNOG,Actinobacteria,397,75,106,216,31964 83331 83332 100226 101510 106370 134676 ...


We can use the database to obtain the COG category of other type of annotation for every protein with a Uniprot ID,
```
UniprotID
  -- id_mappings/uniprot/uniprot-15-May-2015.Bacteria.tsv.gz -->
    orthogroup ID (ENOG, COG, KOG)
      -- bactNOG.annotations -->
        COG category (annotation)
```

### List of proteins in orthogroups

**bactNOG.members.tsv**

In [ ]:
eggMemberDf = pd.read_csv(str(eggNOGPath / 'data/bactNOG/bactNOG.members.tsv'), sep='\t', header=None,
                         names='TaxonomicLevel|GroupName|ProteinCount|SpeciesCount|COGFunctionalCategory|ProteinIDs'.split('|'))
eggMemberDf.head()

EggNOG README:

** members files **

Tab delimited files, each line representing:

TaxonomicLevel|GroupName|ProteinCount|SpeciesCount|COGFunctionalCategory|ProteinIDs

ProteinIds format = Taxid.SeqID

In [ ]:
# Stack the orthogroup database
eggMemberStackedDf = split_and_stack_column(eggMemberDf, col='ProteinIDs', colFieldSep=',', chunkSize=2000, copy_df=True)

# Write stacked orthogroup database to file
eggMemberStackedDf.to_csv(str(eggNOGPath / 'data/bactNOG/bactNOG.members_stacked.tsv'), sep='\t')

Run the script `eggNOG_stack_members.py` with 16G of memory.

In [4]:
eggMemberStackedDf = pd.read_csv(str(eggNOGPath / 'data/bactNOG/bactNOG.members_stacked.tsv'),
                                 sep='\t').drop('Unnamed: 0', axis=1)

### Looking at EggNOG phylogenetic and taxonomic tree...

In [18]:
eggSpeciesDf

,#tax id,nog prefix,level name,species count,core species count,periphery species count,adherent species count,members tax ids
0,1,NOG,LUCA,3686,848,1183,1655,394 882 883 1140 1148 2110 2850 3055 3218 3694...
1,57723,aciNOG,Acidobacteria,7,6,1,0,204669 234267 240015 401053 682795 940615 981222
2,204432,acidNOG,Acidobacteriia,4,3,1,0,240015 401053 682795 940615
3,422676,acoNOG,Aconoidasida,8,1,7,0,5821 5825 5833 5850 5855 333668 352914 353154
4,201174,actNOG,Actinobacteria,397,75,106,216,31964 83331 83332 100226 101510 106370 134676 ...
5,5338,agaNOG,Agaricales,3,3,0,0,240176 486041 578458
6,155619,agarNOG,Agaricomycetes,4,3,1,0,5306 240176 486041 578458
7,5794,apiNOG,Apicomplexa,13,2,11,0,5811 5821 5825 5833 5850 5855 333668 352914 35...
8,28211,aproNOG,Proteobacteria_alpha,290,90,102,98,394 52598 66084 78245 82789 89187 114615 13772...
9,200783,aquNOG,Aquificae,9,8,1,0,123214 204536 224324 380749 436114 608538 6383...


### Build list of all RefSeq and EggNOG species

In [64]:
# Importing list of genome assemblies of the RefSeq database with their taxonomy
# Note: I have included both subspecies and species in the different taxa groups
assemblySummaryRepTaxonomyDf = pd.read_csv(str(taxonomyPath / 'assemblySummaryRepTaxonomyDf.csv'))
assemblySummaryRepTaxonomyDf.set_index('assembly_accession', inplace=True)

In [65]:
# All species included in the EggNOG database
eggSpeciesDf = pd.read_csv(str(eggNOGPath / 'eggnog4_species_by_taxonomic_level.tsv'), sep='\t', header=0)

bactNOGSpeciesTaxidSet = eggSpeciesDf[eggSpeciesDf['nog prefix'] == 'bactNOG']['members tax ids'].iloc[0].split()
bactNOGSpeciesTaxidSet = set([int(x) for x in bactNOGSpeciesTaxidSet])

In [66]:
# write list of protein fasta files
proteinSeqFastaPath = refSeqPath / 'Protein_sequences_fasta'
proteinFastaFilenameList = list(assemblySummaryRepTaxonomyDf['compressedGenomeFilename']\
                                .map(lambda x: re.sub(r'(GCF.+)_genomic\.gbff.*', r'\1_protein.faa', x)))
print(len(proteinFastaFilenameList))
with open(str(proteinSeqFastaPath / 'protein_sequences_fasta_file_list.txt'), 'w') as f:
    f.write("\n".join(proteinFastaFilenameList))

1576


In [11]:
# checking if the genome species was in EggNOG database
print(assemblySummaryRepTaxonomyDf.loc['GCF_900078695.1', 'species_taxid'])
print(assemblySummaryRepTaxonomyDf.loc['GCF_900078695.1', 'taxid'])

123899.0
123899


In [12]:
123899 in bactNOGSpeciesTaxidSet

False

In [13]:
assemblySummaryRepTaxonomyDf[assemblySummaryRepTaxonomyDf['taxid'] == 311403]

,Unnamed: 0,bioproject,biosample,wgs_master,refseq_category,taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,...,species_taxid,subclass_name,subclass_taxid,subfamily_name,subfamily_taxid,subgenus_name,subgenus_taxid,suborder_name,suborder_taxid,subphylum_name,subphylum_taxid,subspecies_name,subspecies_taxid,superkingdom_name,superkingdom_taxid,tribe_name,tribe_taxid
assembly_accession,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GCF_000016265.1,280,PRJNA224116,SAMN02602977,NaN,representative genome,311403,Agrobacterium radiobacter K84,strain=K84,ATCC BAA-868,latest,Complete Genome,Major,Full,2009/01/27,ASM1626v1,NaN,GCA_000016265.1,...,358.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria,2.0,NaN,NaN


### Build list of common species from RefSeq and eggNOG databases

In [71]:
refSeqRankSpeciesSet = set(assemblySummaryRepTaxonomyDf['species_taxid'].unique())

print("Nb of RefSeq assemblies:", len(assemblySummaryRepTaxonomyDf))
print("Nb of RefSeq unique taxid at rank species:", len(refSeqRankSpeciesSet))

Nb of RefSeq assemblies: 1576
Nb of RefSeq unique taxid at rank species: 1516


For each genome in RefSeq, we will include it in the common set if the organism is in the same **species** as the organism in the EggNOG database. This way, we will be able to run the eggmapper only for organisms in the RefSeq database that are very close in taxonomy to at least one organism in the EggNOG database.

We first build a list of **species** taxid for the EggNOG database.

In [68]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

In [69]:
# example
for node in ncbi.get_lineage(1276220):
    if ncbi.get_rank([node])[node] == 'species':
        speciesTaxId = node
        print(speciesTaxId)
    print("rank:", ncbi.get_rank([node]), "rank name:", ncbi.get_taxid_translator([node]))

rank: {1: 'no rank'} rank name: {1: 'root'}
rank: {131567: 'no rank'} rank name: {131567: 'cellular organisms'}
rank: {2: 'superkingdom'} rank name: {2: 'Bacteria'}
rank: {1783272: 'no rank'} rank name: {1783272: 'Terrabacteria group'}
rank: {544448: 'phylum'} rank name: {544448: 'Tenericutes'}
rank: {31969: 'class'} rank name: {31969: 'Mollicutes'}
rank: {186328: 'order'} rank name: {186328: 'Entomoplasmatales'}
rank: {2131: 'family'} rank name: {2131: 'Spiroplasmataceae'}
rank: {2132: 'genus'} rank name: {2132: 'Spiroplasma'}
2145
rank: {2145: 'species'} rank name: {2145: 'Spiroplasma taiwanense'}
rank: {1276220: 'no rank'} rank name: {1276220: 'Spiroplasma taiwanense CT-1'}


In [18]:
bactNOGRankSpeciesTaxidList = []

for taxid in list(bactNOGSpeciesTaxidSet):
    for node in ncbi.get_lineage(taxid):
        if ncbi.get_rank([node])[node] == 'species':
            bactNOGRankSpeciesTaxidList.append((taxid, node))

/home/mweber/local/lib/python3.5/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:224: UserWarning: taxid 508765 was translated into 935198
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/mweber/local/lib/python3.5/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:224: UserWarning: taxid 525327 was translated into 1423757
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/mweber/local/lib/python3.5/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:224: UserWarning: taxid 525364 was translated into 1423799
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/mweber/local/lib/python3.5/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:224: UserWarning: taxid 525366 was translated into 1423814
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/home/mweber/local/lib/python3.5/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:224: UserWarning: taxid 10

Now we have a list of pairs **(taxid in EggNOG, corresponding taxid at the species rank)**. In some cases they are the same and in other cases not, because the EggNOG database uses both ranks in their species list.

Seems that in most cases EggNOG uses the subspecies taxid...

In [ ]:
bactNOGRankSpeciesTaxidList

In [ ]:
bactNOGRankSpeciesTaxidList = [i[1] for i in bactNOGRankSpeciesTaxidList]

In [70]:
print(len(bactNOGRankSpeciesTaxidList))

1713


In [22]:
bactNOGRankSpeciesTaxidList = set(bactNOGRankSpeciesTaxidList)
print(len(bactNOGRankSpeciesTaxidList))

1713


In [26]:
with (analysisEggnogPath / 'bactNOGRankSpeciesTaxidList.json').open('w') as f:
    json.dump(list(bactNOGRankSpeciesTaxidList), f)

In [27]:
with (analysisEggnogPath / 'bactNOGRankSpeciesTaxidList.json').open('r') as f:
    bactNOGRankSpeciesTaxidList = set(json.load(f))

In [28]:
len(bactNOGRankSpeciesTaxidList)

1713

We choose all genomes in the RefSeq database that are in the set of **species** taxid from EggNOG database.

In [72]:
# Define the taxa ids that are common to the EggNOG bacterial database and the full RefSeq database
eggRefSeqCommonRankSpeciesTaxidSet = bactNOGRankSpeciesTaxidList & refSeqRankSpeciesSet

print("Nb of taxid at rank species in RefSeq:", len(refSeqRankSpeciesSet))
print("Nb of taxid at rank species in EggNOG:", len(bactNOGRankSpeciesTaxidList))
print("Nb of taxid at rank species in common", len(eggRefSeqCommonRankSpeciesTaxidSet))

Nb of taxid at rank species in RefSeq: 1516
Nb of taxid at rank species in EggNOG: 1713
Nb of taxid at rank species in common 893


In [73]:
with (analysisEggnogPath / 'eggRefSeqCommonRankSpeciesTaxidSet.json').open('w') as f:
    json.dump([int(j) for j in list(eggRefSeqCommonRankSpeciesTaxidSet)], f)

In [74]:
with (analysisEggnogPath / 'eggRefSeqCommonRankSpeciesTaxidSet.json').open('r') as f:
    eggRefSeqCommonRankSpeciesTaxidSet = set(json.load(f))

In [75]:
# Extract the corresponding genome accession for all species on the list
assemblySummaryCommonSpeciesDf = \
    assemblySummaryRepTaxonomyDf[\
        assemblySummaryRepTaxonomyDf.apply(lambda x: x['species_taxid'] in eggRefSeqCommonRankSpeciesTaxidSet,
                                           axis=1)
                                 ]
print(len(assemblySummaryCommonSpeciesDf))

949


In [76]:
assemblySummaryCommonSpeciesDf.to_csv(str(analysisEggnogPath / 'assemblySummaryCommonSpeciesDf.csv'))

In [77]:
assemblySummaryCommonSpeciesDf = pd.read_csv(str(analysisEggnogPath / 'assemblySummaryCommonSpeciesDf.csv'))
assemblySummaryCommonSpeciesDf.set_index('assembly_accession', inplace=True)
assemblySummaryCommonSpeciesDf.head()

,Unnamed: 0,bioproject,biosample,wgs_master,refseq_category,taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,...,species_taxid,subclass_name,subclass_taxid,subfamily_name,subfamily_taxid,subgenus_name,subgenus_taxid,suborder_name,suborder_taxid,subphylum_name,subphylum_taxid,subspecies_name,subspecies_taxid,superkingdom_name,superkingdom_taxid,tribe_name,tribe_taxid
assembly_accession,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GCF_000005825.2,0,PRJNA224116,SAMN02603086,NaN,representative genome,398511,Bacillus pseudofirmus OF4,strain=OF4,NaN,latest,Complete Genome,Major,Full,2010/12/15,ASM582v2,NaN,GCA_000005825.2,...,79885.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria,2.0,NaN,NaN
GCF_000005845.2,1,PRJNA57779,SAMN02604091,NaN,reference genome,511145,Escherichia coli str. K-12 substr. MG1655,strain=K-12 substr. MG1655,NaN,latest,Complete Genome,Major,Full,2013/09/26,ASM584v2,Univ. Wisconsin,GCA_000005845.2,...,562.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria,2.0,NaN,NaN
GCF_000006605.1,2,PRJNA224116,SAMEA3283089,NaN,representative genome,306537,Corynebacterium jeikeium K411,strain=K411 = NCTC 11915,NaN,latest,Complete Genome,Major,Full,2005/06/27,ASM660v1,NaN,GCA_000006605.1,...,38289.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria,2.0,NaN,NaN
GCF_000006685.1,3,PRJNA224116,SAMN02603994,NaN,representative genome,243161,Chlamydia muridarum str. Nigg,strain=Nigg,NaN,latest,Complete Genome,Major,Full,2003/03/26,ASM668v1,NaN,GCA_000006685.1,...,83560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria,2.0,NaN,NaN
GCF_000006725.1,4,PRJNA224116,SAMN02603773,NaN,representative genome,160492,Xylella fastidiosa 9a5c,NaN,NaN,latest,Complete Genome,Major,Full,2004/06/04,ASM672v1,NaN,GCA_000006725.1,...,2371.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria,2.0,NaN,NaN


In [78]:
# write list of protein fasta files
proteinSeqFastaPath = refSeqPath / 'Protein_sequences_fasta'

assemblySummaryCommonSpeciesDf['protein_fasta_filename'] = \
    assemblySummaryCommonSpeciesDf['compressedGenomeFilename']\
    .map(lambda x: re.sub(r'(GCF.+)_genomic\.gbff.*', r'\1_protein.faa', x))

proteinFastaFilenameList = list(assemblySummaryCommonSpeciesDf['protein_fasta_filename'])
print("Nb of protein fasta files:", len(proteinFastaFilenameList))
with open(str(proteinSeqFastaPath / 'protein_sequences_fasta_file_list_EggNOG_RefSeq_common_species.txt'), 'w') as f:
    f.write("\n".join(proteinFastaFilenameList))

Nb of protein fasta files: 949


We the new definition of genomes, we have 6 more genomes in the common species list.

### Define taxa groups that contain at least 1 genome from common species

In [79]:
taxaDf = pd.read_csv(str(taxonomyPath / 'taxaDf.csv'), index_col=0)
taxaDf.head()

,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,955
4,1,phylum,1224.0,Proteobacteria,697,413
5,1,phylum,1239.0,Firmicutes,259,170
32,3,class,1236.0,Gammaproteobacteria,292,166
23,1,phylum,201174.0,Actinobacteria,226,126


In [80]:
len(taxaDf)

2859

In [81]:
taxaRefSeqFamilyLevel = taxaDf[(taxaDf['n_genomes_in_refseq'] > 0) &
                               (taxaDf['rank_name'] != 'genus')
                              ]
len(taxaRefSeqFamilyLevel)

2133

In the new version, all taxa in taxaDf have at least 1 genome.

In [82]:
taxaRefSeqFamilyLevel = taxaDf
len(taxaRefSeqFamilyLevel)

2859

In [83]:
# Test how many taxon groups have at least 1 genome in the RefSeq database
def find_n_genomes_in_refseq_eggnog_common(taxonRow):
    taxonRank = taxonRow['rank_name']
    taxonId = taxonRow['taxon_id']
    nGenomes = len(assemblySummaryCommonSpeciesDf[assemblySummaryCommonSpeciesDf[taxonRank + '_taxid'] == taxonId])
    return nGenomes

taxaDf['n_genomes_in_refseq_eggnog_common_species'] = taxaDf.apply(find_n_genomes_in_refseq_eggnog_common, axis=1)
taxaDf.sort_values('n_genomes_in_refseq_eggnog_common_species', ascending=False, inplace=True)
taxaDf.head()

,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
4,1,phylum,1224.0,Proteobacteria,697,407
5,1,phylum,1239.0,Firmicutes,259,170
32,3,class,1236.0,Gammaproteobacteria,292,162
23,1,phylum,201174.0,Actinobacteria,226,126


In [84]:
taxaDf.to_csv(str(taxonomyPath / 'taxaDf.csv'))

In [85]:
taxaDf = pd.read_csv(str(taxonomyPath / 'taxaDf.csv'), index_col=0)
print(len(taxaDf))
taxaDf.head(3)

2859


,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
4,1,phylum,1224.0,Proteobacteria,697,407
5,1,phylum,1239.0,Firmicutes,259,170


In [86]:
taxaDf.tail(2)

,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
2064,14,species,83455.0,Myxococcus stipitatus,1,0
2772,14,species,1637999.0,Verrucomicrobia bacterium IMCC26134,1,0


In [87]:
taxaEggnogDf = taxaDf[(taxaDf['n_genomes_in_refseq_eggnog_common_species'] > 0) &
                      (taxaDf['rank_name'] != 'genus')
                     ]
taxaEggnogDf.to_csv(str(analysisEggnogPath / 'taxaEggnogDf.csv'))
print(len(taxaEggnogDf))

1429


In [88]:
taxaEggnogDf = pd.read_csv(str(analysisEggnogPath / 'taxaEggnogDf.csv'), index_col=0)
print(len(taxaEggnogDf))
taxaEggnogDf.head()

1429


,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
4,1,phylum,1224.0,Proteobacteria,697,407
5,1,phylum,1239.0,Firmicutes,259,170
32,3,class,1236.0,Gammaproteobacteria,292,162
23,1,phylum,201174.0,Actinobacteria,226,126


### General ID mapping

### EggNOG mapper

In [89]:
eggnogMapperAnnotationResultPath = analysisEggnogPath / 'Eggnog_mapper_results' / 'HMM_cluster' / 'results'

with (eggnogMapperAnnotationResultPath / 'list_completed_annotations_genomes.txt').open('r') as f:
    listCompletedGenomes = f.readlines()
listCompletedGenomes = [re.search(r'(GCF_\d+(\.\d)?)_*.*', f.strip('\n')).group(1) for f in listCompletedGenomes]
print(len(listCompletedGenomes))
listCompletedGenomes

950


['GCF_000005825.2',
 'GCF_000005845.2',
 'GCF_000006605.1',
 'GCF_000006685.1',
 'GCF_000006725.1',
 'GCF_000006745.1',
 'GCF_000006765.1',
 'GCF_000006785.2',
 'GCF_000006845.1',
 'GCF_000006865.1',
 'GCF_000006905.1',
 'GCF_000006925.2',
 'GCF_000006945.1',
 'GCF_000006965.1',
 'GCF_000006985.1',
 'GCF_000007025.1',
 'GCF_000007045.1',
 'GCF_000007085.1',
 'GCF_000007125.1',
 'GCF_000007145.1',
 'GCF_000007265.1',
 'GCF_000007325.1',
 'GCF_000007365.1',
 'GCF_000007385.1',
 'GCF_000007465.2',
 'GCF_000007485.1',
 'GCF_000007525.1',
 'GCF_000007565.2',
 'GCF_000007605.1',
 'GCF_000007625.1',
 'GCF_000007645.1',
 'GCF_000007705.1',
 'GCF_000007725.1',
 'GCF_000007745.1',
 'GCF_000007765.1',
 'GCF_000007785.1',
 'GCF_000007805.1',
 'GCF_000007825.1',
 'GCF_000007845.1',
 'GCF_000007865.1',
 'GCF_000007905.1',
 'GCF_000007925.1',
 'GCF_000007945.1',
 'GCF_000007985.2',
 'GCF_000008025.1',
 'GCF_000008045.1',
 'GCF_000008165.1',
 'GCF_000008185.1',
 'GCF_000008205.1',
 'GCF_000008305.1',


In [90]:
assemblySummaryIncompleteDf = assemblySummaryCommonSpeciesDf[assemblySummaryCommonSpeciesDf.index.map(lambda x: x not in listCompletedGenomes)]
assemblySummaryIncompleteDf

,Unnamed: 0,bioproject,biosample,wgs_master,refseq_category,taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,...,subclass_name,subclass_taxid,subfamily_name,subfamily_taxid,subgenus_name,subgenus_taxid,suborder_name,suborder_taxid,subphylum_name,subphylum_taxid,subspecies_name,subspecies_taxid,superkingdom_name,superkingdom_taxid,tribe_name,tribe_taxid,protein_fasta_filename
assembly_accession,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [91]:
proteinFastaFilenameList = list(assemblySummaryIncompleteDf['protein_fasta_filename'])
print("Nb of protein fasta files:", len(proteinFastaFilenameList))
with open(str(proteinSeqFastaPath / 'protein_sequences_fasta_file_list_EggNOG_RefSeq_common_species_incomplete.txt'), 'w') as f:
    f.write("\n".join(proteinFastaFilenameList))

Nb of protein fasta files: 0


### Define taxa groups that contain at least 1 genome from common species

In [10]:
taxaDf = pd.read_csv(str(taxonomyPath / 'taxaDf.csv'), index_col=0)
taxaDf.head()

,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
4,1,phylum,1224.0,Proteobacteria,697,407
5,1,phylum,1239.0,Firmicutes,259,170
32,3,class,1236.0,Gammaproteobacteria,292,162
23,1,phylum,201174.0,Actinobacteria,226,126


In [80]:
len(taxaDf)

2859

In [81]:
taxaRefSeqFamilyLevel = taxaDf[(taxaDf['n_genomes_in_refseq'] > 0) &
                               (taxaDf['rank_name'] != 'genus')
                              ]
len(taxaRefSeqFamilyLevel)

2133

In the new version, all taxa in taxaDf have at least 1 genome.

In [82]:
taxaRefSeqFamilyLevel = taxaDf
len(taxaRefSeqFamilyLevel)

2859

In [83]:
# Test how many taxon groups have at least 1 genome in the RefSeq database
def find_n_genomes_in_refseq_eggnog_common(taxonRow):
    taxonRank = taxonRow['rank_name']
    taxonId = taxonRow['taxon_id']
    nGenomes = len(assemblySummaryCommonSpeciesDf[assemblySummaryCommonSpeciesDf[taxonRank + '_taxid'] == taxonId])
    return nGenomes

taxaDf['n_genomes_in_refseq_eggnog_common_species'] = taxaDf.apply(find_n_genomes_in_refseq_eggnog_common, axis=1)
taxaDf.sort_values('n_genomes_in_refseq_eggnog_common_species', ascending=False, inplace=True)
taxaDf.head()

,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
4,1,phylum,1224.0,Proteobacteria,697,407
5,1,phylum,1239.0,Firmicutes,259,170
32,3,class,1236.0,Gammaproteobacteria,292,162
23,1,phylum,201174.0,Actinobacteria,226,126


In [84]:
taxaDf.to_csv(str(taxonomyPath / 'taxaDf.csv'))

In [85]:
taxaDf = pd.read_csv(str(taxonomyPath / 'taxaDf.csv'), index_col=0)
print(len(taxaDf))
taxaDf.head(3)

2859


,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
4,1,phylum,1224.0,Proteobacteria,697,407
5,1,phylum,1239.0,Firmicutes,259,170


### Import annotations results

From [eggmapper documentation](https://github.com/jhcepas/eggnog-mapper/wiki/Results-Interpretation):

<code>[project_name].emapper.annotations</code> file
This file provides final annotations of each query. Tab-delimited columns in the file are:
<ol>
 	<li><code>query_name</code>: query sequence name</li>
 	<li><code>seed_eggNOG_ortholog</code>: best protein match in eggNOG</li>
 	<li><code>seed_ortholog_evalue</code>: best protein match (e-value)</li>
 	<li><code>seed_ortholog_score</code>: best protein match (bit-score)</li>
 	<li><code>predicted_gene_name</code>: Predicted gene name for query sequences</li>
 	<li><code>GO_terms</code>: Comma delimited list of predicted Gene Ontology terms</li>
 	<li><code>KEGG_pathways</code>: Comma delimited list of predicted KEGG pathways</li>
 	<li><code>Annotation_tax_scope</code>: The taxonomic scope used to annotate this query sequence</li>
 	<li><code>Matching_OGs</code>: Comma delimited list of matching eggNOG Orthologous Groups</li>
 	<li><code>best_OG|evalue|score</code>: Best matching Orthologous Groups (only in HMM mode)</li>
 	<li><code>COG functional categories</code>: COG functional category inferred from best matching OG</li>
 	<li><code>eggNOG_HMM_model_annotation</code>: eggNOG functional description inferred from best matching OG</li>
</ol>

In [18]:
analysisEggnogPath

PosixPath('/users/lserrano/mweber/Analysis_data/C-terminal/Eggnog')

In [11]:
eggnogMapperAnnotationResultList = []
eggnogMapperAnnotationErrorList = []
eggnogMapperAnnotationResultPath = analysisEggnogPath / 'Eggnog_mapper_results' / 'HMM_cluster/results2'

#genomeAccession = 'GCF_000006685.1'

# For each genome in the RefSeq database (common species), import eggmapper results
count = 1
for genomeAccession, row in assemblySummaryRepRemainingDf.iterrows():

    print("###", count, "/", len(assemblySummaryRepRemainingDf))
    
    genomeBaseName = re.search(r'^(GCF_.+)_genomic\.gbff\.gz$', row['compressedGenomeFilename']).group(1)
    annotationResultsFilename = genomeBaseName + '.emapper.annotations'
#     print("annotationResultsFilename", annotationResultsFilename)
    annotationResultsFilePath =  eggnogMapperAnnotationResultPath / annotationResultsFilename
    
    if annotationResultsFilePath.is_file():    
        eggnogMapperAnnotationResultDf = \
            pd.read_csv(str(annotationResultsFilePath),
                        sep='\t', header=2).rename(columns={'#query_name':'query_name'})

        # Drop comment lines at the end of the file
        eggnogMapperAnnotationResultDf = eggnogMapperAnnotationResultDf[~eggnogMapperAnnotationResultDf['query_name'].str.contains(r'^#')]

#         eggnogMapperAnnotationResultDf = eggnogMapperAnnotationResultDf[['query_name', 'seed_eggNOG_ortholog', 'seed_ortholog_evalue', 'GO_terms', 'COG cat', 'eggNOG annot']]
        eggnogMapperAnnotationResultDf.rename(columns={'query_name':'refSeqProteinId'}, inplace=True)
        eggnogMapperAnnotationResultDf

        # Find the taxid of the genome
        taxid = assemblySummaryRepRemainingDf.loc[genomeAccession]['taxid']
#         print("taxid", taxid)

        eggnogMapperAnnotationResultDf['genome_accession'] = genomeAccession
        eggnogMapperAnnotationResultDf['taxid'] = taxid
        #print(eggnogMapperAnnotationResultDf.head(2))

        eggnogMapperAnnotationResultList.append(eggnogMapperAnnotationResultDf)
        count += 1
        print("eggmapper annotations imported for genome", genomeAccession,
              "nb of annotations:", len(eggnogMapperAnnotationResultDf))
    else:
        print("ERROR: eggmapper annotation file not found for genome", genomeAccession)
        eggnogMapperAnnotationErrorList.append(genomeAccession)


eggnogMapperAnnotationResultDf = pd.concat(eggnogMapperAnnotationResultList)

### 1 / 627
eggmapper annotations imported for genome GCF_000087965.2 nb of annotations: 2549
### 2 / 627
eggmapper annotations imported for genome GCF_000146065.2 nb of annotations: 1275
### 3 / 627
eggmapper annotations imported for genome GCF_000172635.2 nb of annotations: 3483
### 4 / 627
eggmapper annotations imported for genome GCF_000215085.1 nb of annotations: 3190
### 5 / 627
eggmapper annotations imported for genome GCF_000227665.2 nb of annotations: 1836
### 6 / 627
eggmapper annotations imported for genome GCF_000227685.2 nb of annotations: 2925
### 7 / 627
eggmapper annotations imported for genome GCF_000227705.2 nb of annotations: 3259
### 8 / 627
eggmapper annotations imported for genome GCF_000227745.2 nb of annotations: 3561
### 9 / 627
eggmapper annotations imported for genome GCF_000230695.2 nb of annotations: 3196
### 10 / 627
eggmapper annotations imported for genome GCF_000231385.2 nb of annotations: 3828
### 11 / 627
eggmapper annotations imported for genome GCF_

eggmapper annotations imported for genome GCF_000306785.1 nb of annotations: 4497
### 91 / 627
eggmapper annotations imported for genome GCF_000306885.1 nb of annotations: 583
### 92 / 627
eggmapper annotations imported for genome GCF_000307105.1 nb of annotations: 3661
### 93 / 627
eggmapper annotations imported for genome GCF_000307165.1 nb of annotations: 2068
### 94 / 627
eggmapper annotations imported for genome GCF_000309885.1 nb of annotations: 1943
### 95 / 627
eggmapper annotations imported for genome GCF_000312705.1 nb of annotations: 3092
### 96 / 627
eggmapper annotations imported for genome GCF_000313175.2 nb of annotations: 2089
### 97 / 627
eggmapper annotations imported for genome GCF_000316515.1 nb of annotations: 2726
### 98 / 627
eggmapper annotations imported for genome GCF_000316575.1 nb of annotations: 5064
### 99 / 627
eggmapper annotations imported for genome GCF_000316605.1 nb of annotations: 3610
### 100 / 627
eggmapper annotations imported for genome GCF_0003

eggmapper annotations imported for genome GCF_000442645.1 nb of annotations: 2201
### 178 / 627
eggmapper annotations imported for genome GCF_000444875.1 nb of annotations: 5644
### 179 / 627
eggmapper annotations imported for genome GCF_000444995.1 nb of annotations: 2983
### 180 / 627
eggmapper annotations imported for genome GCF_000445425.4 nb of annotations: 660
### 181 / 627
eggmapper annotations imported for genome GCF_000447675.1 nb of annotations: 2072
### 182 / 627
eggmapper annotations imported for genome GCF_000455605.1 nb of annotations: 2351
### 183 / 627
eggmapper annotations imported for genome GCF_000463355.1 nb of annotations: 1650
### 184 / 627
eggmapper annotations imported for genome GCF_000468615.2 nb of annotations: 3563
### 185 / 627
eggmapper annotations imported for genome GCF_000471025.2 nb of annotations: 715
### 186 / 627
eggmapper annotations imported for genome GCF_000471965.1 nb of annotations: 516
### 187 / 627
eggmapper annotations imported for genome G

eggmapper annotations imported for genome GCF_000706725.1 nb of annotations: 3335
### 251 / 627
eggmapper annotations imported for genome GCF_000709555.1 nb of annotations: 236
### 252 / 627
eggmapper annotations imported for genome GCF_000723165.1 nb of annotations: 4422
### 253 / 627
eggmapper annotations imported for genome GCF_000723465.1 nb of annotations: 1533
### 254 / 627
eggmapper annotations imported for genome GCF_000723505.1 nb of annotations: 2079
### 255 / 627
eggmapper annotations imported for genome GCF_000724485.1 nb of annotations: 2749
### 256 / 627
eggmapper annotations imported for genome GCF_000724605.1 nb of annotations: 1443
### 257 / 627
eggmapper annotations imported for genome GCF_000724625.1 nb of annotations: 3116
### 258 / 627
eggmapper annotations imported for genome GCF_000725345.1 nb of annotations: 4077
### 259 / 627
eggmapper annotations imported for genome GCF_000725365.1 nb of annotations: 3095
### 260 / 627
eggmapper annotations imported for genome

eggmapper annotations imported for genome GCF_000828655.1 nb of annotations: 1856
### 339 / 627
eggmapper annotations imported for genome GCF_000828675.1 nb of annotations: 2002
### 340 / 627
eggmapper annotations imported for genome GCF_000828715.1 nb of annotations: 2992
### 341 / 627
eggmapper annotations imported for genome GCF_000828815.1 nb of annotations: 587
### 342 / 627
eggmapper annotations imported for genome GCF_000828835.1 nb of annotations: 3036
### 343 / 627
eggmapper annotations imported for genome GCF_000828855.1 nb of annotations: 405
### 344 / 627
eggmapper annotations imported for genome GCF_000828895.1 nb of annotations: 2165
### 345 / 627
eggmapper annotations imported for genome GCF_000828915.1 nb of annotations: 2145
### 346 / 627
eggmapper annotations imported for genome GCF_000828975.1 nb of annotations: 2481
### 347 / 627
eggmapper annotations imported for genome GCF_000829195.1 nb of annotations: 3061
### 348 / 627
eggmapper annotations imported for genome 

eggmapper annotations imported for genome GCF_001011155.1 nb of annotations: 4054
### 425 / 627
eggmapper annotations imported for genome GCF_001013905.1 nb of annotations: 5619
### 426 / 627
eggmapper annotations imported for genome GCF_001017435.1 nb of annotations: 4590
### 427 / 627
eggmapper annotations imported for genome GCF_001017775.2 nb of annotations: 3757
### 428 / 627
eggmapper annotations imported for genome GCF_001020955.1 nb of annotations: 2449
### 429 / 627
eggmapper annotations imported for genome GCF_001020985.1 nb of annotations: 2256
### 430 / 627
eggmapper annotations imported for genome GCF_001021025.1 nb of annotations: 2029
### 431 / 627
eggmapper annotations imported for genome GCF_001021045.1 nb of annotations: 2211
### 432 / 627
eggmapper annotations imported for genome GCF_001021065.1 nb of annotations: 1836
### 433 / 627
eggmapper annotations imported for genome GCF_001021085.1 nb of annotations: 851
### 434 / 627
eggmapper annotations imported for genome

eggmapper annotations imported for genome GCF_001308265.1 nb of annotations: 2740
### 513 / 627
eggmapper annotations imported for genome GCF_001310085.1 nb of annotations: 3411
### 514 / 627
eggmapper annotations imported for genome GCF_001310225.1 nb of annotations: 3039
### 515 / 627
eggmapper annotations imported for genome GCF_001310255.1 nb of annotations: 2840
### 516 / 627
eggmapper annotations imported for genome GCF_001314225.1 nb of annotations: 3470
### 517 / 627
eggmapper annotations imported for genome GCF_001314305.1 nb of annotations: 3133
### 518 / 627
eggmapper annotations imported for genome GCF_001314325.1 nb of annotations: 4006
### 519 / 627
eggmapper annotations imported for genome GCF_001314945.1 nb of annotations: 1221
### 520 / 627
eggmapper annotations imported for genome GCF_001318295.1 nb of annotations: 683
### 521 / 627
eggmapper annotations imported for genome GCF_001318345.1 nb of annotations: 4497
### 522 / 627
eggmapper annotations imported for genome

In [12]:
len(eggnogMapperAnnotationErrorList)

42

In [13]:
print(len(eggnogMapperAnnotationResultDf))
eggnogMapperAnnotationResultDf.head(3)

1639599


,refSeqProteinId,seed_eggNOG_ortholog,seed_ortholog_evalue,seed_ortholog_score,predicted_gene_name,GO_terms,KEGG_pathways,Annotation_tax_scope,OGs,bestOG|evalue|score,COG cat,eggNOG annot,genome_accession,taxid
0,WP_012914566.1,190485.XCC0001,4.800000e-265,875.7,DNAA,"GO:0000166,GO:0001882,GO:0001883,GO:0003674,GO...","map02020,map04112",bactNOG[38],"05CI4@bactNOG,0QIVT@gproNOG,16QDA@proNOG,COG05...",05CI4|2.97840944354e-179|600.451416016,L,it binds specifically double-stranded DNA at a...,GCF_000087965.2,380358
1,WP_012914567.1,190486.XAC0002,6.500000e-211,695.8,DNAN,"GO:0003674,GO:0003824,GO:0003887,GO:0005575,GO...","map00230,map00240,map01100,map03030,map03430,m...",bactNOG[38],"05CZ8@bactNOG,0QHTG@gproNOG,16QFW@proNOG,COG05...",05CZ8|5.69544324799e-132|443.953308105,L,"DNA polymerase III is a complex, multichain en...",GCF_000087965.2,380358
2,WP_012914568.1,522373.Smlt0004,2.400000e-188,622.0,RECF,"GO:0000731,GO:0003674,GO:0003676,GO:0003677,GO...",map03440,bactNOG[38],"05C3X@bactNOG,0QJ20@gproNOG,16Q55@proNOG,COG11...",05C3X|2.57774202378e-97|329.962615967,L,it is required for DNA replication and normal ...,GCF_000087965.2,380358


In [14]:
eggnogMapperAnnotationResultDf = \
    pd.concat([
            eggnogMapperAnnotationResultDf[:],
            eggnogMapperAnnotationResultDf[:]['bestOG|evalue|score'].str.split('|', expand=True)\
                .rename(columns={0:'bestOG', 1:'bestOG_evalue', 2:'bestOG_score'})
        ], axis=1).drop(['bestOG|evalue|score'], axis=1)

In [15]:
eggnogMapperAnnotationResultDf2 = eggnogMapperAnnotationResultDf

In [ ]:
eggnogMapperAnnotationResultDf2.head()

In [16]:
eggnogMapperAnnotationResultDf2.to_csv(str(analysisEggnogPath / 'eggnogMapperAnnotationResultDf2.csv'))

In [18]:
eggnogMapperAnnotationResultDf = pd.read_csv(str(analysisEggnogPath / 'eggnogMapperAnnotationResultDf.csv'))
eggnogMapperAnnotationResultDf.drop('Unnamed: 0', axis=1, inplace=True)
eggnogMapperAnnotationResultDf.head()

,refSeqProteinId,seed_eggNOG_ortholog,seed_ortholog_evalue,seed_ortholog_score,predicted_gene_name,GO_terms,KEGG_pathways,Annotation_tax_scope,OGs,COG cat,eggNOG annot,genome_accession,taxid,bestOG,bestOG_evalue,bestOG_score
0,WP_012957018.1,398511.BpOF4_07955,0.000000e+00,1008.6,DNAA,"GO:0000166,GO:0001882,GO:0001883,GO:0003674,GO...","map02020,map04112",bactNOG[38],"04RET@bacNOG,05CI4@bactNOG,0NDJT@firmNOG,COG05...",L,it binds specifically double-stranded DNA at a...,GCF_000005825.2,398511,05CI4,6.694999e-207,691.734497
1,WP_012957019.1,398511.BpOF4_07960,5.000000e-247,814.7,DNAN,"GO:0003674,GO:0003824,GO:0003887,GO:0005575,GO...","map00230,map00240,map01100,map03030,map03430,m...",bactNOG[38],"05CZ8@bactNOG,0ND7B@firmNOG,COG0592@NOG",L,"DNA polymerase III is a complex, multichain en...",GCF_000005825.2,398511,05CZ8,1.862819e-120,405.965668
2,WP_012957020.1,398511.BpOF4_07965,8.700000e-46,145.8,YAAA,NaN,NaN,bactNOG[38],"04XIN@bacNOG,0848D@bactNOG,0NR2T@firmNOG,COG25...",S,S4 domain protein YaaA,GCF_000005825.2,398511,0848D,3.257817e-30,107.957611
3,WP_012957021.1,398511.BpOF4_07970,2.500000e-246,812.6,RECF,"GO:0000731,GO:0005575,GO:0005622,GO:0005623,GO...",map03440,bactNOG[38],"04R5Y@bacNOG,05C3X@bactNOG,0NDA9@firmNOG,COG11...",L,it is required for DNA replication and normal ...,GCF_000005825.2,398511,05C3X,8.569240e-133,447.072968
4,WP_012957022.1,398511.BpOF4_07975,1.100000e-56,179.9,YAAB,NaN,NaN,bactNOG[38],"05WC4@bactNOG,0NNN5@firmNOG,1235W@NOG",S,NaN,GCF_000005825.2,398511,05WC4,1.301090e-27,98.584099


In [22]:
len(eggnogMapperAnnotationResultDf)

2574354

In [23]:
len(eggnogMapperAnnotationResultDf2)

1639599

In [21]:
eggnogMapperAnnotationResultDf3 = pd.concat([eggnogMapperAnnotationResultDf, eggnogMapperAnnotationResultDf2])
eggnogMapperAnnotationResultDf3.to_csv(str(analysisEggnogPath / 'eggnogMapperAnnotationResultDf3.csv'))

In [ ]:
eggnogMapperAnnotationResultDf3 = pd.read_csv(str(analysisEggnogPath / 'eggnogMapperAnnotationResultDf3.csv'),
                                              index_col=0, nrows=None)
eggnogMapperAnnotationResultDf3.head()

As a first indication of the quality of the DIAMOND mapping, we look at the maximum expected value for the seed orthologs, among all sequences.

In [7]:
eggnogMapperAnnotationResultDf3['seed_ortholog_evalue'].max()

8.7e-46

In [8]:
eggnogMapperAnnotationResultDf3['bestOG_evalue'].max()

1.30109005478e-27

Find and group annotations together for each best orthogroup.

In [25]:
def unique_value_or_join_multiple_strings(x):
    xUniqueValues = x.unique()
    if len(xUniqueValues) == 1:
        return xUniqueValues[0]
    else:
        return "|".join([x1 for x1 in xUniqueValues if not pd.isnull(x1)])

In [26]:
eggnogMapperOrthogroupAnnotationDf = \
    eggnogMapperAnnotationResultDf3.groupby(['bestOG'])[['predicted_gene_name', 'KEGG_pathways',
                                                        'COG cat', 'eggNOG annot']]\
                                                      .agg(unique_value_or_join_multiple_strings)

In [27]:
eggnogMapperOrthogroupAnnotationDf.to_csv(str(analysisEggnogPath / 'eggnogMapperOrthogroupAnnotationDf.csv'))

In [10]:
eggnogMapperOrthogroupAnnotationDf = pd.read_csv(str(analysisEggnogPath / 'eggnogMapperOrthogroupAnnotationDf.csv'),
                                                 index_col=0, nrows=10)
eggnogMapperOrthogroupAnnotationDf.head()

,predicted_gene_name,KEGG_pathways,COG cat,eggNOG annot
bestOG,,,,
0000B,ARGD,"map00260,map00300,map00330,map01100,map01110,m...",E,acetylornithine and succinylornithine aminotra...
0000E,ACCC,"map00061,map00253,map00620,map00640,map00720,m...",I,"acetyl-CoA carboxylase, biotin carboxylase"
0000I,NaN,NaN,E,Dienelactone hydrolase family
0000K,CLPP,map04112,O,Cleaves peptides in various proteins in a proc...
0000P,RLUB,NaN,J,Pseudouridine synthase


### Functional enrichment analysis, COG

If we consider clusters of orthologous groups (COG), we can perform an enrichment analysis for protein subsets.

With the EggNOG database we will be able to perform this analysis only at the specific taxonomic levels of the database.

In [197]:
taxonAllCDSDf.groupby('COG cat').size().head(10)

COG cat
C          655
C, O         2
C, P         1
D          109
E          570
E, F         1
E, G, P     14
E, P         7
E, T         5
F          629
dtype: int64

I should check how to handle **multiple COG categories** for the enrichment test. This is a common counfounding factor in gene set analysis. See [Multifunctionality | ermineJ](http://erminej.chibi.ubc.ca/help/tutorials/multifunctionality/) for software that correct for genes that have more functions than others. Also, see

[Park2015] Park, J., Taslim, C., & Lin, S. (2015). BOG: R-package for Bacterium and virus analysis of Orthologous Groups. Computational and Structural Biotechnology Journal, 13, 366–369. http://doi.org/10.1016/j.csbj.2015.05.002

In the following I will assume that most sequences fall only into one COG category and that the influence of sequences with multiple categories is small enough such that we can neglect this effect.

During the COG enrichment analysis, we consider all members of category `Y`, even if such elements can be members of multiple categories. In general, if some elements are members of many categories (annotations), they could bias the enrichment analysis (see references above).

In [199]:
# False discovery rate for the multiple tests correction
alpha_family_wise_error_rate = 0.01
enrichmentTestcolumnName = 'reject_null_multiple_tests_correction_Benjamini-Hochberg_FDR_' + str(alpha_family_wise_error_rate)
groupCol = 'COG cat'
groupName = 'COGGroup'

In [200]:
# Extract list of COG categories
COGList = list(set([s for smult in list(taxonAllCDSDf[groupCol].dropna().unique()) for s in smult.split(', ')]))
COGList

['U',
 'J',
 'T',
 'E',
 'N',
 'M',
 'F',
 'W',
 'V',
 'S',
 'K',
 'G',
 'D',
 'O',
 'L',
 'P',
 'C',
 'I',
 'Q',
 'H']

In [289]:
taxonGroupEnrichDf = \
    enrichment_analysis_cterminal_subset(taxonAllCDSDf, taxonId, groupCol, groupName, COGList, aaTable[:2],
                                         alpha_family_wise_error_rate, enrichmentTestcolumnName)

####### R
####### H


In [291]:
taxonGroupEnrichDf.index

MultiIndex(levels=[[1937961], ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'S', 'T', 'U', 'V', 'Z']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]],
           names=['taxon_id', 'COGGroup_index'])

In [292]:
taxonGroupEnrichDf.columns

MultiIndex(levels=[['H', 'R'], ['count_NOT_cterm_NOT_in_group', 'count_NOT_cterm_in_group', 'count_cterm_NOT_in_group', 'count_cterm_in_group', 'odds_ratio', 'pvalue_fisher', 'reject_null_multiple_tests_correction_Benjamini-Hochberg_FDR_0.01']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]],
           names=['cterm_aa', 'observable'])

In [202]:
taxonGroupEnrichFilename = '{}_enrichment_COG.csv'.format(taxonSuffix)
taxonGroupEnrichDf.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / taxonGroupEnrichFilename))

In [4]:
with open(str(eggNOGPath / 'COG_functional_categories.txt')) as f:
    text = f.read()
    COGCategoriesDict = dict(re.findall(r'\[([A-Z])\]\s+(.*)\s+[$\n]', text))

In [5]:
COGCategoriesDict

{'J': 'Translation, ribosomal structure and biogenesis',
 'A': 'RNA processing and modification',
 'K': 'Transcription',
 'L': 'Replication, recombination and repair',
 'B': 'Chromatin structure and dynamics ',
 'D': 'Cell cycle control, cell division, chromosome partitioning',
 'Y': 'Nuclear structure',
 'V': 'Defense mechanisms',
 'T': 'Signal transduction mechanisms',
 'M': 'Cell wall/membrane/envelope biogenesis',
 'N': 'Cell motility',
 'Z': 'Cytoskeleton',
 'W': 'Extracellular structures',
 'U': 'Intracellular trafficking, secretion, and vesicular transport',
 'O': 'Posttranslational modification, protein turnover, chaperones ',
 'C': 'Energy production and conversion',
 'G': 'Carbohydrate transport and metabolism',
 'E': 'Amino acid transport and metabolism',
 'F': 'Nucleotide transport and metabolism',
 'H': 'Coenzyme transport and metabolism',
 'I': 'Lipid transport and metabolism',
 'P': 'Inorganic ion transport and metabolism',
 'Q': 'Secondary metabolites biosynthesis, tran

In [204]:
# Stack the dataframe for c-terminal AA columns
taxonGroupEnrichStackedDf = taxonGroupEnrichDf.copy()
taxonGroupEnrichStackedDf = taxonGroupEnrichStackedDf.stack(level=['cterm_aa']).reset_index()
taxonGroupEnrichStackedDf.columns.name = ''
taxonGroupEnrichStackedDf.sort_values(['cterm_aa', groupName + '_index'], inplace=True)

# Add COG category description
taxonGroupEnrichStackedDf['COG_cat_description'] = \
    taxonGroupEnrichStackedDf.apply(lambda x:
                                    COGCategoriesDict.get(x['COGGroup_index'], None),
                                    axis=1)

# Select only enriched results
taxonGroupEnrichStackedDf = taxonGroupEnrichStackedDf[taxonGroupEnrichStackedDf[enrichmentTestcolumnName] == True]

# Change column order
taxonGroupEnrichStackedDf = taxonGroupEnrichStackedDf[['taxon_id', 'COGGroup_index', 'COG_cat_description', 'cterm_aa',
       'count_NOT_cterm_NOT_in_group', 'count_NOT_cterm_in_group',
       'count_cterm_NOT_in_group', 'count_cterm_in_group', 'odds_ratio',
       'pvalue_fisher',
       enrichmentTestcolumnName]]

taxonGroupEnrichStackedFilename = '{}_enrichment_COG_stacked.csv'.format(taxonSuffix)
taxonGroupEnrichStackedDf.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / taxonGroupEnrichStackedFilename))

In [205]:
taxonGroupEnrichStackedDf.head()

,taxon_id,COGGroup_index,COG_cat_description,cterm_aa,count_NOT_cterm_NOT_in_group,count_NOT_cterm_in_group,count_cterm_NOT_in_group,count_cterm_in_group,odds_ratio,pvalue_fisher,reject_null_multiple_tests_correction_Benjamini-Hochberg_FDR_0.01
14,544448,J,"Translation, ribosomal structure and biogenesis",H,17165,216,2720,14,0.409024,0.000439059,True
5,544448,E,Amino acid transport and metabolism,R,18751,767,552,45,1.99298,7.44082e-05,True
15,544448,J,"Translation, ribosomal structure and biogenesis",R,16723,658,2580,154,1.51701,1.3368e-05,True


In [206]:
# taxonGroupEnrichFilename = 'enrichment_{}_taxonId_{:d}_{}_{}'\
#                            .format(groupName, int(taxonId), taxonRank, taxonName)
# taxonGroupEnrichDf = pd.read_csv(str(eggNOGPath / 'Analysis_COG_enrichment' / (taxonGroupEnrichFilename + '.csv')),
#                                  header=[0,1], index_col=[0,1])

In [207]:
def enrichment_analysis_cterminal_add_protein_list(taxonAllCDSDf, taxonGroupEnrichDf, groupCol, groupName):
   
    # Convert the enrichment dataframe to a stacked version and
    # Include list of proteins for positive (enriched) orthoGroups into the dataframe.

    # We will include list of proteins for specific orthoGroups into the dataframe,
    # and these will differ for each ctermAA.
    # Therefore, the best solution is to stack all the columns for ctermAA.
    taxonGroupEnrichStackedDf = taxonGroupEnrichDf.copy()
    taxonGroupEnrichStackedDf = taxonGroupEnrichStackedDf.stack(level=['cterm_aa']).reset_index()
    taxonGroupEnrichStackedDf.columns.name = ''
    taxonGroupEnrichStackedDf.sort_values(['cterm_aa', groupName + '_index'], inplace=True)

    taxonAllCDSPositiveDfList = []

    for ctermAA in aaTable:

        # Get list of positive (enriched) orthoGroups
        enrichmentTestDf = pd.DataFrame(taxonGroupEnrichDf.loc[(taxonId), (ctermAA, enrichmentTestcolumnName)])
        enrichmentTestDf.columns = ['reject']

        # Select proteins that are in a positive (enriched) orthoGroup
        taxonAllCDSPositiveDf = pd.merge(taxonAllCDSDf, enrichmentTestDf,
                                         left_on=groupCol, right_index=True, how='left')
        taxonAllCDSPositiveDf = taxonAllCDSPositiveDf[ taxonAllCDSPositiveDf['reject'] == True ]
        taxonAllCDSPositiveDf.drop('reject', axis=1, inplace=True)
        taxonAllCDSPositiveDf.rename(columns={'taxon_taxid':'taxon_id'}, inplace=True)

        taxonAllCDSPositiveDf = taxonAllCDSPositiveDf.reset_index(drop=True)
        taxonAllCDSPositiveDf['cterm_aa'] = ctermAA
        # In order to be able to have a unique multiindex (later), we assign an increasing
        # index to proteins inside groups.
        taxonAllCDSPositiveDf['protein_index'] = taxonAllCDSPositiveDf.groupby(groupCol).cumcount()
        taxonAllCDSPositiveDf.sort_values([groupCol, 'protein_index'], inplace=True)

        taxonAllCDSPositiveDfList.append(taxonAllCDSPositiveDf)


    taxonAllCDSPositiveDf = pd.concat(taxonAllCDSPositiveDfList)

    # Merge the list of proteins with the enrichment dataframe
    print(taxonGroupEnrichStackedDf.columns)
    print(taxonAllCDSPositiveDf.columns)
    taxonGroupEnrichStackedDf = pd.merge(taxonGroupEnrichStackedDf,
                                         taxonAllCDSPositiveDf.rename(columns={groupCol:groupName + '_index'}),
                                         on=['taxon_id', 'cterm_aa', groupName + '_index'], how='outer'
                                         ).sort_values(['taxon_id', 'cterm_aa', groupName + '_index', 'protein_index'])
    return taxonGroupEnrichStackedDf

In [208]:
# taxonGroupEnrichStackedDf = enrichment_analysis_cterminal_add_protein_list(taxonAllCDSDf, taxonGroupEnrichDf,
#                                                                            groupCol, groupName)

In [209]:
# taxonGroupEnrichStackedFilename = 'enrich_{}_taxonRank_{}_taxonName_{}_taxonId_{:0>10d}_stacked'\
#                                   .format(groupName, taxonRank, taxonName, int(taxonId))
# taxonGroupEnrichStackedDf.to_csv(str(OMAFolder / 'TaxonOrthoGroupEnrichment' / (taxonGroupEnrichStackedFilename + '.csv')))

### Functional enrichment analysis, orthologous groups

We will consider the "Best matching Orthologous Groups" as given by the HMM search and fine-grained orthology assignment by the eggmapper software.

In [806]:
print(taxonAllCDSDf[groupCol].isnull().sum(), "/", len(taxonAllCDSDf), "=", taxonAllCDSDf[groupCol].isnull().sum()/len(taxonAllCDSDf))

7428 / 46366 = 0.160203597464


About 33% of protein do not have any OG annotation.

In [492]:
groupCol = 'bestOG'
groupName = 'orthologous_group'

In [807]:
# Extract list of COG categories
orthogroupList = list(taxonAllCDSDf[groupCol].dropna().unique())
len(orthogroupList)

12665

In [821]:
taxonEnrichOrthoGroupDf = \
    enrichment_analysis_cterminal_subset(taxonAllCDSDf, taxonId, groupCol, groupName, orthogroupList[:2], aaTable[:2],
                                         alpha_family_wise_error_rate, enrichmentTestcolumnName)

/users/lserrano/mweber


######## END ANALYSIS JOB ########
####### R
nProteinSubset 7690 nProteinNotSubset 38676
groupIndex 05C0W NjA 5 NjNotA 7685
groupIndex 05CX5 NjA 1 NjNotA 7689
####### H
nProteinSubset 1016 nProteinNotSubset 45350
groupIndex 05C0W NjA 2 NjNotA 1014
groupIndex 05CX5 NjA 0 NjNotA 1016


In [822]:
taxonEnrichOrthoGroupDf.head()

cterm_aa                                                    H  \
observable                       count_NOT_cterm_NOT_in_group   
taxon_id orthologous_group_index                                
1937961  05C0W                                          45316   
         05CX5                                          45342   

cterm_aa                                                   \
observable                       count_NOT_cterm_in_group   
taxon_id orthologous_group_index                            
1937961  05C0W                                         34   
         05CX5                                          8   

cterm_aa                                                   \
observable                       count_cterm_NOT_in_group   
taxon_id orthologous_group_index                            
1937961  05C0W                                       1014   
         05CX5                                       1016   

cterm_aa                                                          \
observable                       count_cterm_in_group odds_ratio   
taxon_id orthologous_group_index                                   
1937961  05C0W                                      2    2.62884   
         05CX5                                      0          0   

cterm_aa                                        \
observable                       pvalue_fisher   
taxon_id orthologous_group_index                 
1937961  05C0W                        0.186312   
         05CX5                               1   

cterm_aa                                                                                            \
observable                       reject_null_multiple_tests_correction_Benjamini-Hochberg_FDR_0.01   
taxon_id orthologous_group_index                                                                     
1937961  05C0W                                                                False                  
         05CX5                                                                False                  

cterm_aa                                                    R  \
observable                       count_NOT_cterm_NOT_in_group   
taxon_id orthologous_group_index                                
1937961  05C0W                                          38645   
         05CX5                                          38669   

cterm_aa                                                   \
observable                       count_NOT_cterm_in_group   
taxon_id orthologous_group_index                            
1937961  05C0W                                         31   
         05CX5                                          7   

cterm_aa                                                   \
observable                       count_cterm_NOT_in_group   
taxon_id orthologous_group_index                            
1937961  05C0W                                       7685   
         05CX5                                       7689   

cterm_aa                                                          \
observable                       count_cterm_in_group odds_ratio   
taxon_id orthologous_group_index                                   
1937961  05C0W                                      5   0.811069   
         05CX5                                      1   0.718448   

cterm_aa                                        \
observable                       pvalue_fisher   
taxon_id orthologous_group_index                 
1937961  05C0W                        0.824074   
         05CX5                               1   

cterm_aa                                                                                            
observable                       reject_null_multiple_tests_correction_Benjamini-Hochberg_FDR_0.01  
taxon_id orthologous_group_index                                                                    
1937961  05C0W                                                                False                 
         05CX5           

In [214]:
taxonEnrichOrthoGroupFilename = '{}_enrichment_orthogroup.csv'.format(taxonSuffix)
taxonEnrichOrthoGroupDf.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / taxonEnrichOrthoGroupFilename))

In [496]:
eggnogMapperOrthogroupAnnotationDf = pd.read_csv(str(analysisEggnogPath / 'eggnogMapperOrthogroupAnnotationDf.csv'))
eggnogMapperOrthogroupAnnotationDf.head()

,bestOG,predicted_gene_name,KEGG_pathways,COG cat,eggNOG annot
0,0000B,ARGD,"map00260,map00300,map00330,map01100,map01110,m...",E,acetylornithine and succinylornithine aminotra...
1,0000E,ACCC,"map00061,map00253,map00620,map00640,map00720,m...",I,"acetyl-CoA carboxylase, biotin carboxylase"
2,0000I,NaN,NaN,E,Dienelactone hydrolase family
3,0000K,CLPP,map04112,O,Cleaves peptides in various proteins in a proc...
4,0000P,RLUB,NaN,J,Pseudouridine synthase


In [499]:
# Stack the dataframe for c-terminal AA columns
taxonEnrichOrthoGroupStackedDf = taxonEnrichOrthoGroupDf.copy()
taxonEnrichOrthoGroupStackedDf = taxonEnrichOrthoGroupStackedDf.stack(level=['cterm_aa']).reset_index()
taxonEnrichOrthoGroupStackedDf.columns.name = ''
taxonEnrichOrthoGroupStackedDf.sort_values(['cterm_aa', groupName + '_index'], inplace=True)

# Add orthogroup annotation
taxonEnrichOrthoGroupStackedDf = \
    pd.merge(taxonEnrichOrthoGroupStackedDf, eggnogMapperOrthogroupAnnotationDf, left_on='orthologous_group_index',
             right_on='bestOG', how='left')

# Select only enriched results
taxonEnrichOrthoGroupStackedDf = taxonEnrichOrthoGroupStackedDf[taxonEnrichOrthoGroupStackedDf[enrichmentTestcolumnName] == True]

# Change column order
taxonEnrichOrthoGroupStackedDf = taxonEnrichOrthoGroupStackedDf[
    ['taxon_id', 'cterm_aa', 'orthologous_group_index', 'predicted_gene_name', 'KEGG_pathways', 'COG cat', 'eggNOG annot',
     'count_NOT_cterm_NOT_in_group', 'count_NOT_cterm_in_group',
     'count_cterm_NOT_in_group', 'count_cterm_in_group', 'odds_ratio',
     'pvalue_fisher',
     enrichmentTestcolumnName]]

taxonEnrichOrthoGroupStackedDf.head()

,taxon_id,cterm_aa,orthologous_group_index,predicted_gene_name,KEGG_pathways,COG cat,eggNOG annot,count_NOT_cterm_NOT_in_group,count_NOT_cterm_in_group,count_cterm_NOT_in_group,count_cterm_in_group,odds_ratio,pvalue_fisher,reject_null_multiple_tests_correction_Benjamini-Hochberg_FDR_0.01
24,1937961,H,07RBZ,SULP|YBAR|RV1739C|HVST,NaN,P,sulfate transporter,45350,1012,0,4,inf,2.29227e-07,True


In [215]:
taxonEnrichOrthoGroupStackedFilename = '{}_enrichment_orthogroup_stacked.csv'.format(taxonSuffix)
taxonEnrichOrthoGroupStackedDf.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / taxonEnrichOrthoGroupStackedFilename))

In [217]:
taxonEnrichOrthoGroupStackedDf.head()

,taxon_id,cterm_aa,orthologous_group_index,predicted_gene_name,KEGG_pathways,COG cat,eggNOG annot,count_NOT_cterm_NOT_in_group,count_NOT_cterm_in_group,count_cterm_NOT_in_group,count_cterm_in_group,odds_ratio,pvalue_fisher,reject_null_multiple_tests_correction_Benjamini-Hochberg_FDR_0.01
2299,544448,R,07RPZ,OPPC|OPPC-1,map02010,E,transport system permease protein,19298,800,5,12,57.894,8.8859e-14,True
2377,544448,R,07SI6,NaN,map02010,E,transport system permease protein,19302,808,1,4,95.5545,1.27597e-05,True
2713,544448,R,080EB,RBSC-2,NaN,G,"ABC transporter, permease",19302,795,1,17,412.747,2.95835e-23,True
2761,544448,R,0810X,CODA,"map00240,map00330,map00791,map01100,map01120","F, J",deaminase,19301,807,2,5,59.7924,2.07839e-06,True
2917,544448,R,084BG,RPSU,map03010,J,30S ribosomal protein S21,19303,805,0,7,inf,1.7039e-10,True


## Run enrichment computation for all taxa

### Enrichment analysis

### C-terminal biases for COG groups

In [111]:
taxaEggnogDf = pd.read_csv(str(analysisEggnogPath / 'taxaEggnogDf.csv'), index_col=0)
taxaEggnogDf.reset_index(inplace=True)
taxaEggnogDf[taxaEggnogDf['taxon_name'] == 'Firmicutes']

,index,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
2,5,1,phylum,1239.0,Firmicutes,259,170


In [114]:
taxaEggnogDf[taxaEggnogDf['taxon_name'] == 'Bacteria']
taxaEggnogDf.iloc[0]

index                                                   0
rank_index                                              0
rank_name                                    superkingdom
taxon_id                                                2
taxon_name                                       Bacteria
n_genomes_in_refseq                                  1582
n_genomes_in_refseq_eggnog_common_species             949
Name: 0, dtype: object

In [112]:
taxaEggnogDf.iloc[2]

index                                                 5
rank_index                                            1
rank_name                                        phylum
taxon_id                                           1239
taxon_name                                   Firmicutes
n_genomes_in_refseq                                 259
n_genomes_in_refseq_eggnog_common_species           170
Name: 2, dtype: object

## Import enrichment results

+ Import results for each taxon
+ Concatenate dataframes
+ Write to disk

In [118]:
taxaEggnogDf = pd.read_csv(str(analysisEggnogPath / 'taxaEggnogDf.csv'), index_col=0).reset_index(drop=True)
print(len(taxaEggnogDf))
taxaEggnogDf.head()

1429


,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
1,1,phylum,1224.0,Proteobacteria,697,407
2,1,phylum,1239.0,Firmicutes,259,170
3,3,class,1236.0,Gammaproteobacteria,292,162
4,1,phylum,201174.0,Actinobacteria,226,126


In [ ]:
taxonGroupEnrichDfList = []
taxonGroupEnrichStackedDfList = []
taxonEnrichOrthoGroupDfList = []
taxonEnrichOrthoGroupStackedDfList = []
OGerrorList = []
OGStackederrorList = []

for i, taxaGroup in taxaEggnogDf.iterrows():
    
    print("### ", i, "/", len(taxaEggnogDf), "taxonGroup:", taxaGroup)
    iTaxon = i
    taxonRank = taxonGroup['rank_name']
    taxonName = taxonGroup['taxon_name']
    taxonId = int(taxonGroup['taxon_id'])
    
    taxonSuffix = simplify_prefix('taxaId_{:d}'.format(taxonId).strip("'"))
    
    filePath = analysisEggnogPath / 'Analysis_COG_enrichment' / '{}_enrichment_COG.csv'.format(taxonSuffix)
    if filePath.is_file():
        taxonGroupEnrichDf = pd.read_csv(str(filePath), header=[0,1], index_col=[0,1])
        taxonGroupEnrichDfList.append(taxonGroupEnrichDf)
    else:
        print("ERROR: file", str(filePath.name), "not found.")
        
    filePath = analysisEggnogPath / 'Analysis_COG_enrichment' / '{}_enrichment_COG_stacked.csv'.format(taxonSuffix)
    if filePath.is_file():
        taxonGroupEnrichStackedDf = pd.read_csv(str(filePath), index_col=0)
        taxonGroupEnrichStackedDfList.append(taxonGroupEnrichStackedDf)
    else:
        print("ERROR: file", str(filePath.name), "not found.")

    filePath = analysisEggnogPath / 'Analysis_COG_enrichment' / '{}_enrichment_orthogroup.csv'.format(taxonSuffix)
    if filePath.is_file():
        taxonEnrichOrthoGroupDf = pd.read_csv(str(filePath), header=[0,1], index_col=[0,1])
        taxonEnrichOrthoGroupDfList.append(taxonEnrichOrthoGroupDf)
    else:
        print("ERROR: file", str(filePath.name), "not found.")
        OGerrorList.append(taxaGroup)
        
    filePath = analysisEggnogPath / 'Analysis_COG_enrichment' / '{}_enrichment_orthogroup_stacked.csv'.format(taxonSuffix)
    if filePath.is_file():
        taxonEnrichOrthoGroupStackedDf = pd.read_csv(str(filePath), index_col=0)
        taxonEnrichOrthoGroupStackedDfList.append(taxonEnrichOrthoGroupStackedDf)
    else:
        print("ERROR: file", str(filePath.name), "not found.")
        OGStackederrorList.append(taxaGroup)

In [12]:
taxonGroupEnrichDfFull = pd.concat(taxonGroupEnrichDfList)
print("taxonGroupEnrichDfList:", len(taxonGroupEnrichDfList), "/", len(taxaGroupCommonEggNOGRefSeqList))

taxonGroupEnrichStackedDfFull = pd.concat(taxonGroupEnrichStackedDfList)
print("taxonGroupEnrichStackedDfList:", len(taxonGroupEnrichStackedDfList), "/", len(taxaGroupCommonEggNOGRefSeqList))

taxonEnrichOrthoGroupDfFull = pd.concat(taxonEnrichOrthoGroupDfList)
print("taxonEnrichOrthoGroupDfList:", len(taxonEnrichOrthoGroupDfList), "/", len(taxaGroupCommonEggNOGRefSeqList))

taxonEnrichOrthoGroupStackedDfFull = pd.concat(taxonEnrichOrthoGroupStackedDfList)
print("taxonEnrichOrthoGroupStackedDfList:", len(taxonEnrichOrthoGroupStackedDfList), "/", len(taxaGroupCommonEggNOGRefSeqList))

taxonGroupEnrichDfList: 434 / 434
taxonGroupEnrichStackedDfList: 434 / 434
taxonEnrichOrthoGroupDfList: 433 / 434
taxonEnrichOrthoGroupStackedDfList: 433 / 434


In [13]:
taxonGroupEnrichDfFull.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / 'taxonGroupEnrichDfFull.csv'))
taxonGroupEnrichStackedDfFull.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / 'taxonGroupEnrichStackedDfFull.csv'))
taxonEnrichOrthoGroupDfFull.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / 'taxonEnrichOrthoGroupDfFull.csv'))
taxonEnrichOrthoGroupStackedDfFull.to_csv(str(analysisEggnogPath / 'Analysis_COG_enrichment' / 'taxonEnrichOrthoGroupStackedDfFull.csv'))

2017.02.24 09:50

We get, at the moment,
```
taxonGroupEnrichDfList: 434 / 434
taxonGroupEnrichStackedDfList: 434 / 434
taxonEnrichOrthoGroupDfList: 421 / 434
taxonEnrichOrthoGroupStackedDfList: 421 / 434
```

## Import c-terminal bias results

In [4]:
%aimport analysis_summarize_results
from analysis_summarize_results import gather_cterm_bias_taxon_results

In [5]:
with open(str(taxonomyPath / 'taxaGroupsInRefSeqNGenomes.json'), 'r') as f:
    taxaGroupsInRefSeqNGenomes = json.load(f)

taxaGroupsInRefSeqNGenomes = [g for g in taxaGroupsInRefSeqNGenomes if g[1] > 0 and g[0][2] != 'genus']
taxaGroupsInRefSeq = [g[0] for g in taxaGroupsInRefSeqNGenomes if g[1] > 0 and g[0][2] != 'genus']
print(len(taxaGroupsInRefSeq))
print(len(taxaGroupsInRefSeqNGenomes))
taxaGroupsInRefSeqNGenomes[:5]

499
499


[[[3, 0, 'phylum', 'Proteobacteria', 1224.0], 691],
 [[143, 1, 'class', 'Gammaproteobacteria', 1236.0], 288],
 [[4, 0, 'phylum', 'Firmicutes', 1239.0], 259],
 [[32, 0, 'phylum', 'Actinobacteria', 201174.0], 226],
 [[144, 1, 'class', 'Actinobacteria', 1760.0], 211]]

In [6]:
oddsRatioCtermTaxaDf = gather_cterm_bias_taxon_results(taxaGroupsInRefSeq)

Importing C-terminal bias results for all taxon groups.
###  1 / 499 taxonGroup: [3, 0, 'phylum', 'Proteobacteria', 1224.0]
###  2 / 499 taxonGroup: [143, 1, 'class', 'Gammaproteobacteria', 1236.0]
###  3 / 499 taxonGroup: [4, 0, 'phylum', 'Firmicutes', 1239.0]
###  4 / 499 taxonGroup: [32, 0, 'phylum', 'Actinobacteria', 201174.0]
###  5 / 499 taxonGroup: [144, 1, 'class', 'Actinobacteria', 1760.0]
###  6 / 499 taxonGroup: [145, 1, 'class', 'Alphaproteobacteria', 28211.0]
###  7 / 499 taxonGroup: [156, 1, 'class', 'Bacilli', 91061.0]
###  8 / 499 taxonGroup: [0, 0, 'phylum', 'Bacteroidetes', 976.0]
###  9 / 499 taxonGroup: [146, 1, 'class', 'Betaproteobacteria', 28216.0]
###  10 / 499 taxonGroup: [161, 1, 'class', 'Clostridia', 186801.0]
###  11 / 499 taxonGroup: [231, 2, 'order', 'Bacillales', 1385.0]
###  12 / 499 taxonGroup: [263, 2, 'order', 'Enterobacterales', 91347.0]
###  13 / 499 taxonGroup: [226, 2, 'order', 'Rhizobiales', 356.0]
###  14 / 499 taxonGroup: [255, 2, 'order', 'Co

## Gather all results by taxon

I gather all results by taxon and c-terminal AA.

taxon, bias in c-terminal aa (-1 position), enrichment in COG, enrichment in orthogroup.

In [7]:
oddsRatioCtermTaxaStackedDf = \
    oddsRatioCtermTaxaDf.drop(['position from terminus', 'terminus'], axis=1)\
    .rename(columns={'taxaid':'taxon_id'})\
    .set_index(['taxon_id', 'taxon_name', 'taxon_rank', 'n_seq', 'n_genomes', 'observable'])\
    .stack()
    
oddsRatioCtermTaxaStackedDf.head(10)

taxon_id  taxon_name      taxon_rank  n_seq      n_genomes  observable   
1224      Proteobacteria  phylum      2165400.0  691        count       R    208748.0
                                                                        H     66152.0
                                                                        K    172536.0
                                                                        D    106100.0
                                                                        E    139547.0
                                                                        S    147598.0
                                                                        T     53226.0
                                                                        N     78816.0
                                                                        Q     95791.0
                                                                        C     19087.0
dtype: float64

In [8]:
oddsRatioCtermTaxaStackedDf.index.names = ['taxon_id', 'taxon_name', 'taxon_rank', 'n_seq', 'n_genomes', 'observable', 'cterm_aa']
oddsRatioCtermTaxaStackedDf.name = 'value'
oddsRatioCtermTaxaStackedDf = \
    oddsRatioCtermTaxaStackedDf.unstack(level=['observable'])\
                               .reset_index(level=['n_seq', 'n_genomes'])\
                               .rename(columns=(lambda x: 'cterm_bias_' + x))\
                               .reset_index(level=['taxon_name', 'taxon_rank'])
#                                .drop('count', axis=1)\
                
oddsRatioCtermTaxaStackedDf.head()

observable           taxon_name taxon_rank  cterm_bias_n_seq  \
taxon_id cterm_aa                                              
29       R         Myxococcales      order           91404.0   
         H         Myxococcales      order           91404.0   
         K         Myxococcales      order           91404.0   
         D         Myxococcales      order           91404.0   
         E         Myxococcales      order           91404.0   

observable         cterm_bias_n_genomes  cterm_bias_count  \
taxon_id cterm_aa                                           
29       R                           13           15288.0   
         H                           13            2076.0   
         K                           13            3962.0   
         D                           13            3398.0   
         E                           13            5420.0   

observable         cterm_bias_log2OddsRatio  cterm_bias_oddsRatio  \
taxon_id cterm_aa                                                   
29       R                         1.176471              2.260232   
         H                         0.086519              1.061805   
         K                         0.807761              1.750493   
         D                        -0.596094              0.661543   
         E                        -0.153837              0.898857   

observable         cterm_bias_pvalue  
taxon_id cterm_aa                     
29       R              0.000000e+00  
         H              7.429050e-03  
         K             2.293645e-220  
         D             6.121232e-140  
         E              1.232448e-14

In [ ]:
taxonGroupEnrichStackedDfFull2 = \
    taxonGroupEnrichStackedDfFull\
        .drop(enrichmentTestcolumnName, axis=1)\
        .set_index(['taxon_id', 'cterm_aa'])\
        .rename(columns={'COGGroup_index':'group_index'})
#         .rename(columns=(lambda x: 'COG_enrichment_' + x))

taxonGroupEnrichStackedDfFull2['enrichment_analysis'] = 'COG'


# CORRECT MISTAKE IN THE SCRIPT, SWAP COUNT COLUMNS
# cols
# cols = taxonGroupEnrichStackedDfFull2.columns.tolist()
# cols[4], cols[5] = cols[5], cols[4]
# taxonGroupEnrichStackedDfFull2 = taxonGroupEnrichStackedDfFull2[cols]
# taxonGroupEnrichStackedDfFull2.rename(columns={'count_NOT_cterm_in_group':'count_cterm_NOT_in_group',
#                                                'count_cterm_NOT_in_group':'count_NOT_cterm_in_group',
#                                               })


cols = taxonGroupEnrichStackedDfFull2.columns.tolist()
cols = cols[-1:] + cols[:-1]
taxonGroupEnrichStackedDfFull2 = taxonGroupEnrichStackedDfFull2[cols]
taxonGroupEnrichStackedDfFull2.reset_index(inplace=True)
taxonGroupEnrichStackedDfFull2.head(3)

In [ ]:
taxonEnrichOrthoGroupStackedDfFull2 = \
    taxonEnrichOrthoGroupStackedDfFull\
        .drop(enrichmentTestcolumnName, axis=1)\
        .set_index(['taxon_id', 'cterm_aa'])\
        .rename(columns={'orthologous_group_index':'group_index',
                         'predicted_gene_name':'OG_predicted_gene_name',
                         'KEGG_pathways':'OG_KEGG_pathways',
                         'COG cat':'OG_COG_cat',
                         'eggNOG annot':'OG_eggNOG_annot'
                        })

        
# CORRECT MISTAKE IN THE SCRIPT, SWAP COUNT COLUMNS
# cols = taxonEnrichOrthoGroupStackedDfFull2.columns.tolist()
# cols
# print(cols[6])
# print(cols[7])
# cols[6], cols[7] = cols[7], cols[6]
# taxonEnrichOrthoGroupStackedDfFull2 = taxonEnrichOrthoGroupStackedDfFull2[cols]
# taxonEnrichOrthoGroupStackedDfFull2.rename(columns={'count_NOT_cterm_in_group':'count_cterm_NOT_in_group',
#                                                'count_cterm_NOT_in_group':'count_NOT_cterm_in_group',
#                                               })
        

taxonEnrichOrthoGroupStackedDfFull2['enrichment_analysis'] = 'OG'
cols = taxonEnrichOrthoGroupStackedDfFull2.columns.tolist()
cols = cols[-1:] + cols[:-1]
taxonEnrichOrthoGroupStackedDfFull2 = taxonEnrichOrthoGroupStackedDfFull2[cols]
taxonEnrichOrthoGroupStackedDfFull2.reset_index(inplace=True)
taxonEnrichOrthoGroupStackedDfFull2.head(3)

In [ ]:
enrichGatheredDf = pd.merge(taxonGroupEnrichStackedDfFull2,
                            taxonEnrichOrthoGroupStackedDfFull2,
                            on=['taxon_id', 'cterm_aa', 'enrichment_analysis', 'group_index',
                                'count_NOT_cterm_NOT_in_group', 'count_NOT_cterm_in_group',
                                'count_cterm_NOT_in_group', 'count_cterm_in_group', 'odds_ratio',
                                'pvalue_fisher'],
                            how='outer')

# Add log2 odds ratio for easier plotting
enrichGatheredDf['log2_odds_ratio'] = np.log2(enrichGatheredDf['odds_ratio'])

enrichGatheredDf = enrichGatheredDf[[
    'taxon_id',
    'cterm_aa',
    'enrichment_analysis',
    'group_index',
    'count_NOT_cterm_NOT_in_group',
    'count_NOT_cterm_in_group',
    'count_cterm_NOT_in_group',
    'count_cterm_in_group',
    'odds_ratio',
    'log2_odds_ratio',
    'pvalue_fisher',
    'COG_cat_description',
    'OG_predicted_gene_name',
    'OG_KEGG_pathways',
    'OG_COG_cat',
    'OG_eggNOG_annot']]

In [ ]:
ctermGatheredDf = pd.merge(oddsRatioCtermTaxaStackedDf.reset_index(), enrichGatheredDf,
                           on=['taxon_id', 'cterm_aa'], how='outer')

# Add proportion
ctermGatheredDf['cterm_in_group_proportion'] = \
    ctermGatheredDf['count_cterm_in_group'] / (ctermGatheredDf['count_cterm_in_group'] + ctermGatheredDf['count_cterm_NOT_in_group'])

ctermGatheredDf = ctermGatheredDf[[
    'taxon_id',
    'taxon_name',
    'taxon_rank',
    'cterm_aa',
    'cterm_bias_n_seq',
    'cterm_bias_n_genomes',
    'cterm_bias_count',
    'cterm_bias_log2OddsRatio',
    'cterm_bias_oddsRatio',
    'cterm_bias_pvalue',
    'enrichment_analysis',
    'group_index',
    'count_NOT_cterm_NOT_in_group',
    'count_NOT_cterm_in_group',
    'count_cterm_NOT_in_group',
    'count_cterm_in_group',
    'cterm_in_group_proportion',
    'odds_ratio',
    'log2_odds_ratio',
    'pvalue_fisher',
    'COG_cat_description',
    'OG_predicted_gene_name',
    'OG_KEGG_pathways',
    'OG_COG_cat',
    'OG_eggNOG_annot']]
ctermGatheredDf.sort_values(['cterm_aa', 'cterm_bias_n_seq', 'taxon_rank', 'taxon_name', 'enrichment_analysis',
                             'odds_ratio'],
                            ascending=[True, False, True, True, True, True],
                            inplace=True)
print(len(ctermGatheredDf))

-----

In [ ]:
ctermGatheredDf.to_csv(str(analysisEggnogPath / 'ctermGatheredDf.csv'))

In [3]:
ctermGatheredDf = pd.read_csv(str(analysisEggnogPath / 'ctermGatheredDf.csv'))
ctermGatheredDf.drop('Unnamed: 0', axis=1, inplace=True)
ctermGatheredDf.head()

,taxon_id,taxon_name,taxon_rank,cterm_aa,cterm_bias_n_seq,cterm_bias_n_genomes,cterm_bias_count,cterm_bias_log2OddsRatio,cterm_bias_oddsRatio,cterm_bias_pvalue,enrichment_analysis,group_index,count_NOT_cterm_NOT_in_group,count_NOT_cterm_in_group,count_cterm_NOT_in_group,count_cterm_in_group,cterm_in_group_proportion,odds_ratio,log2_odds_ratio,pvalue_fisher,COG_cat_description,OG_predicted_gene_name,OG_KEGG_pathways,OG_COG_cat,OG_eggNOG_annot
0,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,W,1110051.0,184.0,165788.0,2.0,0.000012,0.072778,-3.780347,4.702118e-09,Extracellular structures,NaN,NaN,NaN,NaN
1,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,Z,1110145.0,90.0,165787.0,3.0,0.000018,0.223207,-2.163545,2.855893e-03,Cytoskeleton,NaN,NaN,NaN,NaN
2,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,U,1085557.0,24678.0,163848.0,1942.0,0.011714,0.521376,-0.939605,1.033719e-198,"Intracellular trafficking, secretion, and vesi...",NaN,NaN,NaN,NaN
3,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,M,1045675.0,64560.0,159418.0,6372.0,0.038434,0.647398,-0.627274,5.106927e-257,Cell wall/membrane/envelope biogenesis,NaN,NaN,NaN,NaN
4,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,N,1094780.0,15455.0,163881.0,1909.0,0.011515,0.825154,-0.277264,8.300436e-16,Cell motility,NaN,NaN,NaN,NaN


-----

In [ ]:
# HTML rendering with Pandas style

#     ctermGatheredDfFocusStyled = \
#         ctermGatheredDfFocus\
#             .style.apply(lambda x: background_gradient(x, cmap=colormap, vmin=-3, vmax=+3),
#                          subset='log2 odds ratio')
#     ctermGatheredDfFocusStyled

#     with (analysisEggnogPath / 'ctermGatheredDfFocus.html').open('w') as f:
#         f.write(ctermGatheredDfFocusStyled.render())

In [26]:
import mwTools.plot
colormap = mwTools.plot.get_divergent_color_map()

In [27]:
from xlsxpandasformatter.xlsxpandasformatter import FormatedWorksheet

In [31]:
for ctermAA in ['T', 'P', 'M', 'K', 'R']:
# for ctermAA in aaTable:
# for ctermAA in ['T']:

    ### Looking at COG enrichment
    
    ctermGatheredDfFocus = \
        ctermGatheredDf[
            (ctermGatheredDf['cterm_aa'] == ctermAA)
            & (ctermGatheredDf['enrichment_analysis'] == 'COG')
    #             & (ctermGatheredDf['cterm_bias_oddsRatio'] < 0.6) &
    #             (ctermGatheredDf['cterm_bias_n_seq'] > 20e3) &
    #             & ((ctermGatheredDf['odds_ratio'] > 2) |
    #              (ctermGatheredDf['odds_ratio'] < 0.5))
        ]\
        .rename(columns=(lambda x: re.sub(r'_', r' ', x)))

    df = ctermGatheredDfFocus
    nRows = len(df)
    nCols = len(df.columns)
    writer = pd.ExcelWriter(str(analysisEggnogPath / 'ctermGatheredDf_cterm_{}_COG_enrichment.xlsx'.format(ctermAA)),
                            engine='xlsxwriter', options={'nan_inf_to_errors': True})
    df.to_excel(writer, index=False, sheet_name='sheet1')
    workbook = writer.book
    worksheet = writer.sheets['sheet1']

    formattedWorksheet = FormatedWorksheet(worksheet, workbook, df, hasIndex=False)

    headerFormat={'valign':'bottom', 'bold':True, 'bottom':2, 'text_wrap':True}
    headerHeight = 80
    formattedWorksheet.format_header(headerFormat=headerFormat, rowHeight=headerHeight)

    formattedWorksheet.format_background_colormap('cterm bias log2OddsRatio', colormap, vmin=-3, vmax=+3)
    formattedWorksheet.format_background_colormap('log2 odds ratio', colormap, vmin=-3, vmax=+3)
    formattedWorksheet.format_add_separation_border_between_groups('taxon id')

    numFormatScientific = '0.0E+00'
    numFormatFloat2digits = 0x02
    numFormatInteger = 0x01
    colPatternFormatList = [
        (r'(pvalue)', numFormatScientific),
        (r'(proportion)', numFormatScientific),
        (r'([oO]ddsRatio)|(odds ratio)', numFormatFloat2digits),
        (r'(cterm bias n seq)|(count)', numFormatInteger)
    ]
    formattedWorksheet.format_numeric_cols(colPatternFormatList)

    # Apply the color to the column
    colIndexCOGdescription = formattedWorksheet.df.columns.get_loc('COG cat description')
    colIndexCOGdescription

    def format_COG_cat_description(row):

        formatSeries = pd.Series(data=[dict() for _ in range(len(row))], index=row.index)

        if row['group index'] in ['J', 'C']:
            # Translation, ribosomal structure and biogenesis
            # Energy production and conversion
            formatSeries[colIndexCOGdescription]['font_color'] = '#0e4179'
        elif row['group index'] == 'B':
            # 'Chromatin structure and dynamics '
            formatSeries[colIndexCOGdescription]['font_color'] = '#7c0722'

        return formatSeries

    formattedWorksheet.format_pandas_apply(format_COG_cat_description, axis=1)

    formattedWorksheet.apply_format_table()
    writer.save()
    
    ### Looking at OG enrichment

#     ctermGatheredDfFocus = \
#         ctermGatheredDf[
#             (ctermGatheredDf['cterm_aa'] == ctermAA)
#             & (ctermGatheredDf['enrichment_analysis'] == 'OG')
# #             & (ctermGatheredDf['cterm_bias_oddsRatio'] < 0.6)
# #             & (ctermGatheredDf['cterm_bias_n_seq'] > 10e3)
#     #         ((ctermGatheredDf['odds_ratio'] > 2) |
#     #          (ctermGatheredDf['odds_ratio'] < 0.5))
#         ]\
#         .rename(columns=(lambda x: re.sub(r'_', r' ', x)))
    
#     df = ctermGatheredDfFocus
#     nRows = len(df)
#     nCols = len(df.columns)
#     writer = pd.ExcelWriter(str(analysisEggnogPath / 'ctermGatheredDf_cterm_{}_OG_enrichment.xlsx'.format(ctermAA)),
#                             engine='xlsxwriter', options={'nan_inf_to_errors': True})
#     df.to_excel(writer, index=False, sheet_name='sheet1')
#     workbook = writer.book
#     worksheet = writer.sheets['sheet1']
    
#     formattedWorksheet = FormatedWorksheet(worksheet, workbook, df, hasIndex=False)
#     formattedWorksheet.format_header(headerFormat=headerFormat, rowHeight=headerHeight)
#     formattedWorksheet.format_background_colormap('cterm bias log2OddsRatio', colormap, vmin=-3, vmax=+3)
#     formattedWorksheet.format_add_separation_border_between_groups('taxon id')
#     formattedWorksheet.format_numeric_cols(colPatternFormatList)
#     formattedWorksheet.apply_format_table()
#     writer.save()
#     print("ctermAA", ctermAA, " done.")

In [40]:
df = ctermGatheredDf[(ctermGatheredDf['enrichment_analysis'] == 'COG') &
                     (ctermGatheredDf['cterm_bias_n_genomes'] >= 1) &
                     (ctermGatheredDf['cterm_aa'] == 'S') &
#                      (ctermGatheredDf['taxon_rank'] == 'order') &
                     (ctermGatheredDf['cterm_bias_n_seq'] > 1e4) &
                     (ctermGatheredDf['log2_odds_ratio'].map(lambda x: x < -1.5 or x > +1.5))
                    ]
df

,taxon_id,taxon_name,taxon_rank,cterm_aa,cterm_bias_n_seq,cterm_bias_n_genomes,cterm_bias_count,cterm_bias_log2OddsRatio,cterm_bias_oddsRatio,cterm_bias_pvalue,enrichment_analysis,group_index,count_NOT_cterm_NOT_in_group,count_NOT_cterm_in_group,count_cterm_NOT_in_group,count_cterm_in_group,cterm_in_group_proportion,odds_ratio,log2_odds_ratio,pvalue_fisher,COG_cat_description,OG_predicted_gene_name,OG_KEGG_pathways,OG_COG_cat,OG_eggNOG_annot
103071,1224,Proteobacteria,phylum,S,2165400.0,691,147598.0,0.260771,1.198119,0.000000e+00,COG,A,1189455.0,284.0,86282.0,4.0,0.000046,0.194164,-2.364650,0.000030,RNA processing and modification,NaN,NaN,NaN,NaN
103088,1236,Gammaproteobacteria,class,S,907108.0,288,62501.0,0.217833,1.162985,2.670244e-277,COG,A,471226.0,185.0,34269.0,3.0,0.000088,0.222986,-2.164976,0.001976,RNA processing and modification,NaN,NaN,NaN,NaN
106544,85023,Microbacteriaceae,family,S,18534.0,7,1725.0,0.801149,1.742488,3.928653e-92,COG,D,7558.0,57.0,789.0,17.0,0.021092,2.856959,1.514480,0.000489,"Cell cycle control, cell division, chromosome ...",NaN,NaN,NaN,NaN
106667,775,Rickettsiaceae,family,S,15896.0,15,1231.0,0.199970,1.148674,4.766884e-06,COG,F,9648.0,222.0,842.0,4.0,0.004728,0.206458,-2.276078,0.000090,Nucleotide transport and metabolism,NaN,NaN,NaN,NaN
106716,81852,Enterococcaceae,family,S,15758.0,6,879.0,-0.104689,0.930006,3.675565e-02,COG,J,9655.0,612.0,579.0,11.0,0.018644,0.299719,-1.738315,0.000004,"Translation, ribosomal structure and biogenesis",NaN,NaN,NaN,NaN
106719,1903410,Pectobacteriaceae,family,S,15451.0,4,949.0,0.062500,1.044274,1.994609e-01,COG,J,10275.0,614.0,694.0,14.0,0.019774,0.337584,-1.566681,0.000004,"Translation, ribosomal structure and biogenesis",NaN,NaN,NaN,NaN
106839,1763524,Isosphaeraceae,family,S,10771.0,2,1077.0,0.898511,1.864141,3.583733e-71,COG,I,3281.0,51.0,383.0,19.0,0.047264,3.191471,1.674221,0.000092,Lipid transport and metabolism,NaN,NaN,NaN,NaN


## C-terminal bias for each COG category group

## Final set of EggNOG annotated sequences

In our pipeline, we gather the data in the following way.

In [4]:
analysisEggnogPath

PosixPath('/users/lserrano/mweber/Analysis_data/C-terminal/Eggnog')

In [5]:
taxaEggnogDf = pd.read_csv(str(analysisEggnogPath / 'taxaEggnogDf.csv'), index_col=0).reset_index(drop=True)
print(len(taxaEggnogDf))
taxaEggnogDf.head()

1429


,rank_index,rank_name,taxon_id,taxon_name,n_genomes_in_refseq,n_genomes_in_refseq_eggnog_common_species
0,0,superkingdom,2.0,Bacteria,1582,949
1,1,phylum,1224.0,Proteobacteria,697,407
2,1,phylum,1239.0,Firmicutes,259,170
3,3,class,1236.0,Gammaproteobacteria,292,162
4,1,phylum,201174.0,Actinobacteria,226,126


In [6]:
taxonId = 2

In [41]:
# Importing list of genome assemblies with their taxonomy in the RefSeq - EggNOG common set of species
assemblySummaryCommonSpeciesDf = pd.read_csv(str(analysisEggnogPath / 'assemblySummaryCommonSpeciesDf.csv'))

taxonSuffix = simplify_prefix('taxaId_{:d}'.format(taxonId).strip("'"))
print("taxonSuffix", taxonSuffix)

# Import all protein sequences in taxon group from file
multispeciesAllCDSDf_nr = pd.read_csv(str(analysisCtermDataPath / 'Taxonomy' / taxonSuffix / (taxonSuffix + '_multispeciesAllCDSDf_nr.csv')))
print("len(multispeciesAllCDSDf_nr)", len(multispeciesAllCDSDf_nr))

taxonSuffix taxaId_2
len(multispeciesAllCDSDf_nr) 4871983


In [42]:
# Choose protein sequences from genomes that are in the common set of species RefSeq - EggNOG
multispeciesAllCDSDf_nr2 = pd.merge(multispeciesAllCDSDf_nr,
                                    assemblySummaryCommonSpeciesDf.set_index('assembly_accession')[['species_taxid', 'taxid']],
                                    left_on='genome_accession', right_index=True, how='left')
taxonAllCDSEggNOGSpeciesDf = multispeciesAllCDSDf_nr2.dropna(subset=['taxid'], axis=0)
nSeqTaxonRefSeq = len(multispeciesAllCDSDf_nr2)
nSeqTaxonRefSeqEggNOGCommon = len(taxonAllCDSEggNOGSpeciesDf)
print("nSeqTaxonRefSeqEggNOGCommon:", nSeqTaxonRefSeqEggNOGCommon, "nSeqTaxonRefSeq", nSeqTaxonRefSeq,
      "ratio", nSeqTaxonRefSeqEggNOGCommon/nSeqTaxonRefSeq)
print("Nb of subspecies in taxon group, RefSeq:", len(set(multispeciesAllCDSDf_nr2['taxid'].dropna())))
print("Nb of taxid at rank species in taxon group, RefSeq:", len(set(multispeciesAllCDSDf_nr2['species_taxid'].dropna())))
print("Nb of subspecies in taxon group, common:", len(set(taxonAllCDSEggNOGSpeciesDf['taxid'])))
print("Nb of taxid at rank species, common:", len(set(taxonAllCDSEggNOGSpeciesDf['species_taxid'])))

# Import eggnog-mapper protein annotations
eggnogMapperAnnotationResultDf = pd.read_csv(str(analysisEggnogPath / 'eggnogMapperAnnotationResultDf.csv'))
eggnogMapperAnnotationResultDf.drop('Unnamed: 0', axis=1, inplace=True)
eggnogMapperOrthogroupAnnotationDf = pd.read_csv(str(analysisEggnogPath / 'eggnogMapperOrthogroupAnnotationDf.csv'))

# Merging eggnog-mapper annotations with the taxonAllCDS
taxonAllCDSDf = pd.merge(taxonAllCDSEggNOGSpeciesDf.reset_index(),
                         eggnogMapperAnnotationResultDf,
                         on='refSeqProteinId', how='left', suffixes=['_refseq', '_eggnog'])
taxonAllCDSDf.drop('index', axis=1, inplace=True)
taxonAllCDSDf.drop('genome_accession_eggnog', axis=1, inplace=True)
taxonAllCDSDf.rename(columns={'genome_accession_refseq':'genome_accession'}, inplace=True)
taxonAllCDSDf.drop('taxid_eggnog', axis=1, inplace=True)
taxonAllCDSDf.rename(columns={'taxid_refseq':'taxid'}, inplace=True)
print("taxonAllCDSDf:\n", taxonAllCDSDf.head(), "\n")

print("Nb of protein sequences with EggNOG orthogroup annotation:",
      taxonAllCDSDf['COG cat'].notnull().sum(), "/", len(taxonAllCDSDf), "=",
      taxonAllCDSDf['COG cat'].notnull().sum()/len(taxonAllCDSDf))

nSeqTaxonRefSeqEggNOGCommon: 2866659 nSeqTaxonRefSeq 4871983 ratio 0.5883967575420522
Nb of subspecies in taxon group, RefSeq: 946
Nb of taxid at rank species in taxon group, RefSeq: 893
Nb of subspecies in taxon group, common: 946
Nb of taxid at rank species, common: 893
taxonAllCDSDf:
   refSeqProteinId  proteinGI               species_name genome_accession  \
0  WP_012960546.1  502725562  Bacillus pseudofirmus OF4  GCF_000005825.2   
1  WP_012960083.1  502725099  Bacillus pseudofirmus OF4  GCF_000005825.2   
2  WP_012957129.1  502722145  Bacillus pseudofirmus OF4  GCF_000005825.2   
3  WP_012957437.1  502722453  Bacillus pseudofirmus OF4  GCF_000005825.2   
4  WP_012958269.1  502723285  Bacillus pseudofirmus OF4  GCF_000005825.2   

                                              DNASeq stopCodon  \
0  ATGTCTAGAAAGAAGTACAACCCTTTCATTATGACGATCATGATTT...       TAA   
1  TTGACAATCTATCAAGATATTATACGTATATTTGAAGAGAATTCTT...       TAA   
2  TTGACAGGTCAACTTGTTCAGTATGGACGCCACCGCCAACGTAGAA...    

In [43]:
taxonAllCDSDf.to_csv(analysisEggnogPath / 'taxonAllCDSDf_bacteria.csv')

In [9]:
ctermGatheredDf = pd.read_csv(str(analysisEggnogPath / 'ctermGatheredDf.csv'))
ctermGatheredDf.drop('Unnamed: 0', axis=1, inplace=True)
ctermGatheredDf.head()

,taxon_id,taxon_name,taxon_rank,cterm_aa,cterm_bias_n_seq,cterm_bias_n_genomes,cterm_bias_count,cterm_bias_log2OddsRatio,cterm_bias_oddsRatio,cterm_bias_pvalue,enrichment_analysis,group_index,count_NOT_cterm_NOT_in_group,count_NOT_cterm_in_group,count_cterm_NOT_in_group,count_cterm_in_group,cterm_in_group_proportion,odds_ratio,log2_odds_ratio,pvalue_fisher,COG_cat_description,OG_predicted_gene_name,OG_KEGG_pathways,OG_COG_cat,OG_eggNOG_annot
0,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,W,1110051.0,184.0,165788.0,2.0,0.000012,0.072778,-3.780347,4.702118e-09,Extracellular structures,NaN,NaN,NaN,NaN
1,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,Z,1110145.0,90.0,165787.0,3.0,0.000018,0.223207,-2.163545,2.855893e-03,Cytoskeleton,NaN,NaN,NaN,NaN
2,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,U,1085557.0,24678.0,163848.0,1942.0,0.011714,0.521376,-0.939605,1.033719e-198,"Intracellular trafficking, secretion, and vesi...",NaN,NaN,NaN,NaN
3,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,M,1045675.0,64560.0,159418.0,6372.0,0.038434,0.647398,-0.627274,5.106927e-257,Cell wall/membrane/envelope biogenesis,NaN,NaN,NaN,NaN
4,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.0,COG,N,1094780.0,15455.0,163881.0,1909.0,0.011515,0.825154,-0.277264,8.300436e-16,Cell motility,NaN,NaN,NaN,NaN


In [14]:
df = ctermGatheredDf[ctermGatheredDf['cterm_aa'] == 'A']
df = df[df['enrichment_analysis'] == 'COG']
df[df['log2_odds_ratio'] < -2.5]

,taxon_id,taxon_name,taxon_rank,cterm_aa,cterm_bias_n_seq,cterm_bias_n_genomes,cterm_bias_count,cterm_bias_log2OddsRatio,cterm_bias_oddsRatio,cterm_bias_pvalue,enrichment_analysis,group_index,count_NOT_cterm_NOT_in_group,count_NOT_cterm_in_group,count_cterm_NOT_in_group,count_cterm_in_group,cterm_in_group_proportion,odds_ratio,log2_odds_ratio,pvalue_fisher,COG_cat_description,OG_predicted_gene_name,OG_KEGG_pathways,OG_COG_cat,OG_eggNOG_annot
0,1224,Proteobacteria,phylum,A,2165400.0,691,282036.0,0.288529,1.221394,0.000000e+00,COG,W,1110051.0,184.0,165788.0,2.0,0.000012,0.072778,-3.780347,4.702118e-09,Extracellular structures,NaN,NaN,NaN,NaN
33,1236,Gammaproteobacteria,class,A,907108.0,288,97855.0,0.127226,1.092192,8.434666e-146,COG,W,452391.0,91.0,53201.0,0.0,0.000000,0.000000,-inf,9.264406e-05,Extracellular structures,NaN,NaN,NaN,NaN
107,28216,Betaproteobacteria,class,A,350719.0,108,57119.0,0.493002,1.407370,0.000000e+00,COG,W,167203.0,46.0,32959.0,1.0,0.000030,0.110284,-3.180706,4.709665e-03,Extracellular structures,NaN,NaN,NaN,NaN
1179,80840,Burkholderiales,order,A,244299.0,66,43671.0,0.582566,1.497510,0.000000e+00,COG,W,108457.0,37.0,24033.0,0.0,0.000000,0.000000,-inf,9.661445e-04,Extracellular structures,NaN,NaN,NaN,NaN
1443,28221,Deltaproteobacteria,class,A,237872.0,58,24943.0,-0.072842,0.950763,3.515301e-14,COG,Z,144836.0,67.0,17234.0,0.0,0.000000,0.000000,-inf,1.042534e-03,Cytoskeleton,NaN,NaN,NaN,NaN
4500,29,Myxococcales,order,A,91404.0,13,11604.0,-0.021559,0.985167,1.338548e-01,COG,Z,40153.0,38.0,6175.0,0.0,0.000000,0.000000,-inf,7.264395e-03,Cytoskeleton,NaN,NaN,NaN,NaN
6009,1161,Nostocales,order,A,55974.0,11,3276.0,-0.498777,0.707707,1.655817e-90,COG,U,13790.0,158.0,916.0,0.0,0.000000,0.000000,-inf,7.727454e-05,"Intracellular trafficking, secretion, and vesi...",NaN,NaN,NaN,NaN
7341,1162,Nostocaceae,family,A,33780.0,7,1996.0,-0.494139,0.709985,9.906988e-55,COG,U,13790.0,158.0,916.0,0.0,0.000000,0.000000,-inf,7.727454e-05,"Intracellular trafficking, secretion, and vesi...",NaN,NaN,NaN,NaN
8331,2092,Mycoplasmataceae,family,A,23456.0,33,775.0,-0.673758,0.626872,6.314348e-43,COG,O,14920.0,414.0,562.0,2.0,0.003546,0.128251,-2.962953,6.936636e-05,"Posttranslational modification, protein turnov...",NaN,NaN,NaN,NaN
8336,2085,Mycoplasmatales,order,A,23456.0,33,775.0,-0.673758,0.626872,6.314348e-43,COG,O,14920.0,414.0,562.0,2.0,0.003546,0.128251,-2.962953,6.936636e-05,"Posttranslational modification, protein turnov...",NaN,NaN,NaN,NaN
